## 環境構築

### ログ初期化

In [ ]:
import logging
import os
import subprocess
import sys

if os.path.exists("debug.log"):
    os.remove("debug.log")

def custom_format(record):
    match record.levelno:
        case logging.DEBUG:
            level = "🟦"
        case logging.INFO:
            level = "🟩"
        case logging.WARNING:
            level = "🟨"
        case logging.ERROR:
            level = "🟥"
        case logging.CRITICAL:
            level = "🛑"
    return f"{level} {record.getMessage()}"

logger = logging.getLogger()

for handler in logger.handlers:
    logger.removeHandler(handler)

formatter = logging.Formatter()
formatter.format = custom_format

file_handler = logging.FileHandler("debug.log")
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

stream_handler = logging.StreamHandler()
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)

logger.setLevel(logging.DEBUG)

NVIDIA_SMI = subprocess.run(["nvidia-smi"], capture_output=True, text=True).stdout
logging.info(NVIDIA_SMI)
logging.info(f"Python {sys.version}")

### インストール

In [ ]:
# 1) PyTorchとTransformersのインストール
%pip install torch==2.4.0

# 2) vLLMのインストール
%pip install vllm==0.6.3

# 3) Flash Attentionのインストール
# 2.8.3はundefined symbolエラーが発生するため2.7.3をインストール
# https://github.com/Dao-AILab/flash-attention/issues/1832
# %pip install "https://github.com/Dao-AILab/flash-attention/releases/download/v2.7.3/flash_attn-2.7.3+cu12torch2.6cxx11abiFALSE-cp312-cp312-linux_x86_64.whl" --no-build-isolation
%pip install flash-attn --no-build-isolation

In [ ]:
%cd /workspaces/tinyzero/TinyZero
%pip install -e .

In [ ]:
%pip install wandb

### インポート

In [ ]:
from codetiming import Timer
from contextlib import contextmanager
from dataclasses import dataclass, field
from datasets import Dataset, load_dataset
from enum import Enum
from filelock import FileLock
from flash_attn.bert_padding import pad_input, unpad_input, index_first_axis, rearrange
from omegaconf import DictConfig, open_dict, OmegaConf
from pprint import pprint
from random import randint, seed, choice
from ray.experimental.state.api import get_actor
from ray.util import list_named_actors
from ray.util.placement_group import placement_group, PlacementGroup
from ray.util.scheduling_strategies import PlacementGroupSchedulingStrategy, NodeAffinitySchedulingStrategy
from torch import nn
from torch import optim
from torch.distributed.device_mesh import init_device_mesh
from torch.distributed.fsdp import FullyShardedDataParallel as FSDP, ShardingStrategy, CPUOffload, MixedPrecision, StateDictType, FullStateDictConfig
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import AutoConfig, AutoModelForTokenClassification, AutoModelForCausalLM
from typing import Dict, List, Any, Tuple, Type
import argparse
import ast
import hashlib
import hydra
import itertools
import numpy as np
import operator
import os
import random
import ray
import re
import shutil
import string
import tempfile
import time
import torch
import torch.distributed
import uuid
import warnings

In [ ]:
from verl import DataProto
from verl.models.registry import check_model_support_rmpad
from verl.models.transformers.monkey_patch import apply_monkey_patch
from verl.protocol import pad_dataproto_to_divisor, unpad_dataproto
from verl.single_controller.base import WorkerGroup, ResourcePool, ClassWithInitArgs, Worker
from verl.single_controller.base.decorator import register, Dispatch
from verl.single_controller.ray.base import create_colocated_worker_cls
from verl.trainer.ppo import core_algos
from verl.utils import hf_tokenizer
from verl.utils.dataset.rl_dataset import RLHFDataset, collate_fn
from verl.utils.debug import log_gpu_memory_usage
from verl.utils.flops_counter import FlopsCounter
# from verl.utils.fs import copy_local_path_from_hdfs
from verl.utils.fsdp_utils import get_fsdp_wrap_policy, offload_fsdp_grad, init_fn, get_init_weight_context_manager, offload_fsdp_optimizer, offload_fsdp_param_and_grad, load_fsdp_optimizer, load_fsdp_param_and_grad
from verl.utils.hdfs_io import copy, makedirs
from verl.utils.import_utils import import_external_libs
from verl.utils.model import compute_position_id_with_mask, LambdaLayer, print_model_size, squeeze, update_model_config
from verl.utils.seqlen_balancing import get_seqlen_balanced_partitions, log_seqlen_unbalance, rearrange_micro_batches, get_reverse_idx
from verl.utils.torch_dtypes import PrecisionType
from verl.utils.torch_functional import get_constant_schedule_with_warmup
from verl.utils.tracking import Tracking
from verl.utils.ulysses import ulysses_pad_and_slice_inputs, gather_outpus_and_unpad
from verl.workers.actor import DataParallelPPOActor
from verl.workers.critic import DataParallelPPOCritic
from verl.workers.rollout import HFRollout
from verl.workers.rollout.vllm_rollout import vLLMRollout
from verl.workers.sharding_manager import BaseShardingManager, FSDPVLLMShardingManager
from verl.workers.sharding_manager.fsdp_ulysses import FSDPUlyssesShardingManager
import verl.utils.hdfs_io as hdfs_io
import verl.utils.torch_functional as verl_F

## データセット構築

[Countdown-Task-3to4][1]をダウンロードし、チャット形式のデータセットを作成する:

```sh
python ./examples/data_preprocess/countdown.py --local_dir countdown
```

[1]: https://huggingface.co/datasets/Jiayi-Pan/Countdown-Tasks-3to4

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('--local_dir', default='~/data/countdown')
parser.add_argument('--hdfs_dir', default=None)
parser.add_argument('--num_samples', type=int, default=100000)
parser.add_argument('--num_operands', type=int, default=6)
parser.add_argument('--max_target', type=int, default=1000)
parser.add_argument('--min_number', type=int, default=1)
parser.add_argument('--max_number', type=int, default=100)
parser.add_argument('--train_size', type=int, default=327680)
parser.add_argument('--test_size', type=int, default=1024)
parser.add_argument('--template_type', type=str, default='base')

args = parser.parse_args(["--local_dir", "countdown"])

# デバッグ用に上書き
args.train_size = 32 
args.test_size = 32

data_source = 'countdown'
TRAIN_SIZE = args.train_size
TEST_SIZE = args.test_size

TRAIN_SIZE, TEST_SIZE

In [ ]:
# https://huggingface.co/datasets/Jiayi-Pan/Countdown-Tasks-3to4
raw_dataset = load_dataset('Jiayi-Pan/Countdown-Tasks-3to4', split='train')
logger.info(f"データセットのサイズ: {len(raw_dataset)}")

In [ ]:
assert len(raw_dataset) > TRAIN_SIZE + TEST_SIZE
train_dataset = raw_dataset.select(range(TRAIN_SIZE))
test_dataset = raw_dataset.select(range(TRAIN_SIZE, TRAIN_SIZE + TEST_SIZE))

logger.info(f"訓練データセットのサイズ: {len(train_dataset)}")
logger.info(f"テストデータセットのサイズ: {len(test_dataset)}")
logger.info(f"サンプルデータ: {train_dataset[0]}")

In [ ]:
def make_prefix(dp, template_type):
    """
    データポイントからプロンプトのプレフィックスを作成する

    Args:
        dp (dict): targetとnumsを含むデータポイント
        template_type (str): 使用するテンプレートの種類
            - base: 一般的なベースモデル用
            - qwen-instruct: Qwen Instructモデル用

    Returns:
        str: プロンプトのプレフィックス
    """
    target = dp['target']
    numbers = dp['nums']

    # ベースモデルの場合
    if template_type == 'base':
        # User: {numbers} を使って {target} になる数式を作ってください。
        # 四則演算（+, -, *, /）が使え、各数字は一度だけ使えます。
        # 思考過程は <think> </think> タグで示し、最終的な答えは <answer> </answer> タグで
        # 返してください。例: <answer> (1 + 2) / 3 </answer>。
        # Assistant: 順を追って解いていきます。<think>
        prefix = f"""A conversation between User and Assistant. The user asks a question, and the Assistant solves it. The assistant first thinks about the reasoning process in the mind and then provides the user with the answer.
User: Using the numbers {numbers}, create an equation that equals {target}. You can use basic arithmetic operations (+, -, *, /) and each number can only be used once. Show your work in <think> </think> tags. And return the final answer in <answer> </answer> tags, for example <answer> (1 + 2) / 3 </answer>.
Assistant: Let me solve this step by step.
<think>"""

    # Qwen Instructモデルの場合
    elif template_type == 'qwen-instruct':

        # <|im_start|>system
        # あなたは有能なアシスタントです。まず頭の中で推論プロセスを考え、その後ユーザーに答えを伝えます。
        # <|im_end|>
        # <|im_start|>user
        # {numbers} を使って {target} になる数式を作ってください。
        # 四則演算（+, -, *, /）が使え、各数字は一度だけ使えます。
        # 思考過程は <think> </think> タグで示し、最終的な答えは
        #  <answer> </answer> タグで返してください。
        # 例: <answer> (1 + 2) / 3 </answer>。
        # <|im_end|>
        # <|im_start|>assistant
        # 順を追って解いていきます。
        # <think>
        prefix = f"""<|im_start|>system\nYou are a helpful assistant. You first thinks about the reasoning process in the mind and then provides the user with the answer.<|im_end|>\n<|im_start|>user\n Using the numbers {numbers}, create an equation that equals {target}. You can use basic arithmetic operations (+, -, *, /) and each number can only be used once. Show your work in <think> </think> tags. And return the final answer in <answer> </answer> tags, for example <answer> (1 + 2) / 3 </answer>.<|im_end|>\n<|im_start|>assistant\nLet me solve this step by step.\n<think>"""
    return prefix

make_prefix(train_dataset[0], 'base')

In [ ]:
def make_map_fn(split):
    """
    データセットの前処理に使用するマップ関数を作成する

    Args:
        split (str): データセットの分割名（例: 'train', 'test'）
    Returns:
        function: データポイントを処理するマップ関数
    """

    def process_fn(example, idx):
        """
        データポイントを前処理する

        Args:
            example (dict): データポイント
            idx (int): データポイントのインデックス
        Returns:
            dict: 前処理されたデータポイント
        """

        # データポイントからプロンプトを作成
        question = make_prefix(example, template_type=args.template_type)

        solution = {
            "target": example['target'],
            "numbers": example['nums']
        }

        # 前処理したデータを辞書形式で返す
        data = {
            "data_source": data_source,
            "prompt": [{
                "role": "user",
                "content": question,
            }],
            "ability": "math",
            "reward_model": {
                "style": "rule",
                "ground_truth": solution
            },
            "extra_info": {
                'split': split,
                'index': idx,
            }
        }
        return data
    return process_fn

# 検証
make_map_fn("train")(train_dataset[0], 0)

In [ ]:
# データセットの前処理とParquet形式での保存

local_dir = args.local_dir
OVERWRITE = False

if not os.path.exists(os.path.join(local_dir, "train.parquet")) or OVERWRITE:
    logger.info("train.parquetを作成")
    train_dataset = train_dataset.map(function=make_map_fn('train'), with_indices=True)
    train_dataset.to_parquet(os.path.join(local_dir, 'train.parquet'))
else:
    logger.info(f"既にtrain.parquetが存在するためスキップ: {os.path.join(local_dir, 'train.parquet')}")

if not os.path.exists(os.path.join(local_dir, "test.parquet")) or OVERWRITE:
    logger.info("test.parquetを作成")
    test_dataset = test_dataset.map(function=make_map_fn('test'), with_indices=True)
    test_dataset.to_parquet(os.path.join(local_dir, 'test.parquet'))
else:
    logger.info(f"既にtest.parquetが存在するためスキップ: {os.path.join(local_dir, 'test.parquet')}")

In [ ]:
# Hadoop File System (HDFS) にデータをコピー
# 分散処理用

hdfs_dir = args.hdfs_dir

if hdfs_dir is not None:
    makedirs(hdfs_dir)
    copy(src=local_dir, dst=hdfs_dir) 

## HDFSのユーティリティ関数

TinyZeroでは、HDFS（Hadoop Distributed File System）を使用

リモートのHDFSにあるモデルやデータを、各プロセスがローカルにダウンロードする機能を提供

In [ ]:
# HDFSのファイルパスのプリフィックス
_HDFS_PREFIX = "hdfs://"

In [ ]:
_HDFS_BIN_PATH = shutil.which('hdfs')
logger.info(f"HDFSバイナリのパス: {_HDFS_BIN_PATH}")

In [ ]:
def _is_non_local(path):
    return path.startswith(_HDFS_PREFIX)

In [ ]:
def md5_encode(path: str) -> str:
    return hashlib.md5(path.encode()).hexdigest()

In [ ]:
def get_local_temp_path(hdfs_path: str, cache_dir: str) -> str:
    """
    HDFSパスとキャッシュディレクトリからローカルの一時パスを取得する

    Args:
        hdfs_path (str): HDFS上のファイルパス
        cache_dir (str): ローカルのキャッシュディレクトリ
    Returns:
        str: ローカルの一時ファイルパス
    """
    # ディレクトリの競合を避けるためにhdfs_pathのbase64エンコードを作成
    encoded_hdfs_path = md5_encode(hdfs_path)

    # キャッシュディレクトリ内にエンコードされたHDFSパスのサブディレクトリを作成
    temp_dir = os.path.join(cache_dir, encoded_hdfs_path)
    os.makedirs(temp_dir, exist_ok=True)

    # HDFSパスのベース名をローカルの一時ディレクトリに結合
    dst = os.path.join(temp_dir, os.path.basename(hdfs_path))

    return dst

In [ ]:
def _hdfs_cmd(cmd: str) -> str:
    """
    HDFSコマンドを構築する

    Args:
        cmd (str): HDFSのサブコマンド（例: -cp -f src dst）
    Returns:
        str: 完全なHDFSコマンド（例: hdfs dfs -cp -f src dst）
    """
    return f"{_HDFS_BIN_PATH} dfs {cmd}"

In [ ]:
def _run_cmd(cmd: str, timeout=None):
    """
    システムコマンドを実行する
    タイムアウトは未実装...

    Args:
        cmd (str): 実行するコマンド
        timeout (int, optional): タイムアウト時間（秒）
    Returns:
        int: コマンドの終了ステータス
    """
    return os.system(cmd)

In [ ]:
def _copy(from_path: str, to_path: str, timeout: int = None) -> bool:
    """
    HDFSとローカル間でファイルをコピーする内部関数

    Args:
        from_path (str): コピー元のパス（HDFSまたはローカル）
        to_path (str): コピー先のパス（HDFSまたはローカル）
        timeout (int, optional): コピー操作のタイムアウト時間（秒）。デフォルトはNone。
    Returns:
        bool: コピーが成功した場合はTrue、失敗した場合はFalse
    """

    # コピー先がHDFSの場合
    if to_path.startswith("hdfs"):

        # コピー元がHDFSの場合
        if from_path.startswith("hdfs"):

            # HDFS間のコピーを実行
            returncode = _run_cmd(
                _hdfs_cmd(f"-cp -f {from_path} {to_path}"), timeout=timeout
            )

        # コピー元がローカルの場合
        else:

            # ローカルからHDFSへのコピーを実行
            returncode = _run_cmd(
                _hdfs_cmd(f"-put -f {from_path} {to_path}"), timeout=timeout
            )

    # コピー先がローカルの場合
    else:

        # コピー元がHDFSの場合
        if from_path.startswith("hdfs"):

            # HDFSからローカルへのコピーを実行
            returncode = _run_cmd(
                _hdfs_cmd(f"-get {from_path} {to_path}"), timeout=timeout
            )

        # コピー元がローカルの場合
        else:
            try:
                # ローカルからローカルへのコピーを実行
                shutil.copy(from_path, to_path)
                returncode = 0
            except shutil.SameFileError:
                returncode = 0
            except Exception as e:
                logger.warning(f"copy {from_path} {to_path} failed: {e}")
                returncode = -1

    return returncode == 0

In [ ]:
def copy(src: str, dst: str, **kwargs) -> bool:
    r"""
    HDFSをサポートするコピー関数
    shutil.copy()やshutil.copytree()のように動作する

    Args:
        src (str): コピー元のファイルパス
        dst (str): コピー先のファイルパス
        kwargs: hdfsコピーのためのキーワード引数
    Returns:
        str: コピー先のファイルパス
    """
    logger.info(f"コピー開始: {src} -> {dst}")

    # srcかdstがHDFSパスの場合
    if _is_non_local(src) or _is_non_local(dst):
        return _copy(src, dst)

    # ローカルファイルシステムの場合
    else:
        if os.path.isdir(src):
            return shutil.copytree(src, dst, **kwargs)
        else:
            return shutil.copy(src, dst, **kwargs)

In [ ]:
def copy_local_path_from_hdfs(src: str, cache_dir=None, filelock='.file.lock', verbose=False) -> str:
    """
    HDFS（Hadoop Distributed File System）上のファイルやディレクトリを
    ローカルマシンのキャッシュディレクトリにコピーし、そのローカルパスを返す

    HDFSに対応していないライブラリでも、モデルやデータを透過的に利用可能にする

    Args:
        src (str): HDFS上のソースパス
        cache_dir (str, optional):
            ローカルのキャッシュディレクトリ
            指定しない場合はシステムのデフォルトキャッシュディレクトリを使用
        filelock (str, optional):
            ファイルロックの名前
            デフォルトは'.file.lock'
        verbose (bool, optional):
            コピー時に詳細情報を表示するかどうか
            デフォルトはFalse
    """
    logger.info(f"HDFSからローカルへのコピー開始: {src}")

    # 1) 入力値の検証

    # srcの最後の文字が'/'でないことを確認
    assert src[-1] != '/', f'Make sure the last char in src is not / because it will cause error. Got {src}'

    # 2) HDFSからローカルへのコピー処理

    # ソースがhdfs://で始まる場合
    if _is_non_local(src):
        logger.debug(f"HDFSからローカルにコピー: {src}")

        # キャッシュディレクトリを作成
        if cache_dir is None:
            cache_dir = tempfile.gettempdir()

        os.makedirs(cache_dir, exist_ok=True)

        assert os.path.exists(cache_dir)

        local_path = get_local_temp_path(src, cache_dir)

        # md5ハッシュを使って一意のロックファイル名を生成
        filelock = md5_encode(src) + '.lock'
        lock_file = os.path.join(cache_dir, filelock)

        # ファイルロックを使用して同時コピーを防止
        with FileLock(lock_file=lock_file):
            if not os.path.exists(local_path):
                if verbose:
                    logger.debug(f"コピー中: {src} -> {local_path}")

                # HDFSからローカルにコピー
                copy(src, local_path)

        return local_path
    else:
        logger.debug(f"ローカルパスをそのまま使用: {src}")
        return src

### FSDPワーカー

#### ActorRolloutRefWorker

In [ ]:
class ActorRolloutRefWorker(Worker):
    """
    TinyZero（基盤のVeRL）の分散強化学習において
    学習（Actor）、ロールアウト（Rollout）、参照モデル（Ref Model）の3つの役割を
    1つのワーカーで担う、もしくは個別に扱うためのクラス
    """

    def __init__(self, config: DictConfig, role: str):
        """
        Args:
            config (DictConfig): ワーカーの設定
            role (str): ワーカーの役割
                - 'actor': 学習のみ
                - 'rollout': ロールアウトのみ
                - 'ref': 参照モデルのみ
                - 'actor_rollout': 学習とロールアウトの両方
                - 'actor_rollout_ref': 学習、ロールアウト、参照モデルの全て
        """

        logger.info(f"ActorRolloutRefWorkerを初期化 {config=} {role=}")

        super().__init__()

        self.config = config

        # 1) 分散環境が初期化されていない場合、NCCLバックエンドで初期化

        if not torch.distributed.is_initialized():
            torch.distributed.init_process_group(backend="nccl")

        # 2) FSDP（Fully Sharded Data Parallel）用のデバイスメッシュを構築

        world_size = torch.distributed.get_world_size()

        self.device_mesh = init_device_mesh(
            'cuda',
            mesh_shape=(world_size,),
            mesh_dim_names=['fsdp']
        )

        # 3) Ulysses Sequence Parallel用のデバイスメッシュを構築

        # シーケンス並列化とは、長いシーケンスを複数のデバイスに分割して処理する手法

        self.ulysses_device_mesh = None

        self.ulysses_sequence_parallel_size = \
            self.config.actor.get('ulysses_sequence_parallel_size', 1)

        dp = world_size // self.ulysses_sequence_parallel_size

        if self.ulysses_sequence_parallel_size > 1:
            self.ulysses_device_mesh = init_device_mesh(
                'cuda',
                mesh_shape=(dp, self.ulysses_sequence_parallel_size),
                mesh_dim_names=['dp', 'sp']
            )

        self.ulysses_sharding_manager = FSDPUlyssesShardingManager(
            self.ulysses_device_mesh
        )

        # 4) ワーカーの役割を設定

        self.role = role

        assert self.role in ['actor', 'rollout', 'ref', 'actor_rollout', 'actor_rollout_ref']

        self._is_actor = self.role in ['actor', 'actor_rollout', 'actor_rollout_ref']

        self._is_rollout = self.role in ['rollout', 'actor_rollout', 'actor_rollout_ref']

        self._is_ref = self.role in ['ref', 'actor_rollout_ref']

        self._is_offload_param = False
        self._is_offload_grad = False
        self._is_offload_optimizer = False

        if self._is_actor:
            self._is_offload_param = self.config.actor.fsdp_config.get('param_offload', False)
            self._is_offload_grad = self.config.actor.fsdp_config.get('grad_offload', False)
            self._is_offload_optimizer = self.config.actor.fsdp_config.get('optimizer_offload', False)

        elif self._is_ref:
            # TODO: it seems that manual offload is slowly than FSDP offload
            self._is_offload_param = self.config.ref.fsdp_config.get('param_offload', False)

        # アクターモデルの場合
        if self._is_actor:

            self.config.actor.ppo_mini_batch_size //= \
                (self.device_mesh.shape[0] // self.ulysses_sequence_parallel_size)

            self.config.actor.ppo_micro_batch_size //= \
                (self.device_mesh.shape[0] // self.ulysses_sequence_parallel_size)

            self.config.actor.ppo_mini_batch_size *= self.config.rollout.n

            self.config.actor.ppo_micro_batch_size *= self.config.rollout.n

        # ロールアウトモデルの場合
        if self._is_rollout:

            self.config.rollout.log_prob_micro_batch_size //= \
                (self.device_mesh.shape[0] // self.ulysses_sequence_parallel_size)

            self.config.rollout.log_prob_micro_batch_size *= self.config.rollout.n

        # 参照モデルの場合
        if self._is_ref:

            self.config.ref.log_prob_micro_batch_size //= \
                (self.device_mesh.shape[0] // self.ulysses_sequence_parallel_size)

            self.config.ref.log_prob_micro_batch_size *= self.config.rollout.n

    def _build_model_optimizer(self, model_path, fsdp_config, optim_config, override_model_config, use_remove_padding=False, enable_gradient_checkpointing=False, trust_remote_code=False):
        """
        モデルとオプティマイザを構築する
        init_modelメソッドから呼び出される

        Args:
            model_path (str): 事前学習済みモデルのパス
            fsdp_config (DictConfig): FSDPの設定
            optim_config (DictConfig): オプティマイザの設定
            override_model_config (Dict): モデル設定の上書き用辞書
            use_remove_padding (bool, optional): パディング除去を使用するかどうか
            enable_gradient_checkpointing (bool, optional):
                勾配チェックポイントを有効にするかどうか
            trust_remote_code (bool, optional): リモートコードを信頼するかどうか
        Returns:
            actor_module_fsdp (FSDP): FSDPでラップされたアクターモデル
            actor_optimizer (Optimizer): アクター用オプティマイザ
            actor_lr_scheduler (LRScheduler): アクター用学習率スケジューラ
        """

        # メモリ使用量をログに記録
        log_gpu_memory_usage('Before init from HF AutoModel', logger=logger)

        # モデルをローカルにコピー
        local_path = copy_local_path_from_hdfs(model_path)

        # トークナイザを初期化
        self.tokenizer = hf_tokenizer(local_path, trust_remote_code=trust_remote_code)

        # モデルのデータ型を設定
        torch_dtype = fsdp_config.get('model_dtype', None)

        # モデルのデータ型が指定されていない場合
        if torch_dtype is None:
            # アクターモデルの場合はfloat32、そうでなければbfloat16を使用
            torch_dtype = torch.float32 if self._is_actor else torch.bfloat16

        # 指定されている場合
        else:
            # そのデータ型を使用
            torch_dtype = PrecisionType.to_dtype(torch_dtype)

        # モデル設定を事前学習済みモデルから読み込む
        actor_model_config = AutoConfig.from_pretrained(
            local_path, trust_remote_code=trust_remote_code
        )

        # パディング除去を使用する場合
        if use_remove_padding:

            # モデルがパディング除去をサポートしているか確認
            check_model_support_rmpad(actor_model_config.model_type)

        # Ulysses Sequence Parallelが有効な場合
        if use_remove_padding and self.ulysses_sequence_parallel_size > 1:

            # パディング除去のモンキーパッチを適用
            apply_monkey_patch(actor_model_config, verbose=True)

        # モデル設定の上書き用キーワード引数を準備
        override_config_kwargs = {
            'bos_token_id': self.tokenizer.bos_token_id,
            'eos_token_id': self.tokenizer.eos_token_id,
            'pad_token_id': self.tokenizer.pad_token_id,
        }

        # メソッドの引数で、追加のモデル設定の上書きが指定されている場合は上書き
        override_config_kwargs.update(override_model_config)

        # モデル設定を上書き
        update_model_config(
            actor_model_config,
            override_config_kwargs=override_config_kwargs
        )

        # メインプロセスの場合、モデル設定を表示
        if self.rank == 0:
            logger.info(f'Model config after override: {actor_model_config}')

        # モデルの初期化コンテキストを取得
        init_context = get_init_weight_context_manager(
            use_meta_tensor=not actor_model_config.tie_word_embeddings
        )

        # モデルを初期化
        with init_context(), warnings.catch_warnings():

            # Transformersの警告を無視
            warnings.simplefilter("ignore")

            # アクターモデルを事前学習済みモデルから読み込む
            actor_module = AutoModelForCausalLM.from_pretrained(
                pretrained_model_name_or_path=local_path,
                torch_dtype=torch_dtype,
                config=actor_model_config,
                attn_implementation='flash_attention_2',
                trust_remote_code=trust_remote_code
            )

            # アクターモデルを指定されたデータ型に変換
            actor_module.to(torch_dtype)

            # 勾配チェックポイントを有効にする場合
            if enable_gradient_checkpointing:

                # アクターモデルで勾配チェックポイントを有効化
                actor_module.gradient_checkpointing_enable(
                    gradient_checkpointing_kwargs={'use_reentrant': False}
                )

        # 全プロセスで同期
        torch.distributed.barrier()

        # メインプロセスでモデルサイズを表示
        if self.rank == 0:
            print_model_size(actor_module)

        # メモリ使用量をログに記録
        log_gpu_memory_usage('After init from HF AutoModel', logger=logger)

        # FSDPの設定から混合精度の設定を取得
        mixed_precision_config = fsdp_config.get('mixed_precision', None)

        # 混合精度の設定が指定されている場合
        if mixed_precision_config is not None:

            # モデルのパラメータのデータ型を取得、なければbfloat16を使用
            param_dtype = PrecisionType.to_dtype(
                mixed_precision_config.get('param_dtype', 'bf16')
            )

            # 勾配の集約操作（reduce）のデータ型を取得、なければfloat32を使用
            reduce_dtype = PrecisionType.to_dtype(
                mixed_precision_config.get('reduce_dtype', 'fp32')
            )

            # バッファのデータ型を取得、なければfloat32を使用
            buffer_dtype = PrecisionType.to_dtype(
                mixed_precision_config.get('buffer_dtype', 'fp32')
            )

        # 混合精度の設定が指定されていない場合
        else:
            param_dtype = torch.bfloat16
            reduce_dtype = torch.float32
            buffer_dtype = torch.float32

        # FSDPの混合精度設定を作成
        mixed_precision = MixedPrecision(
            param_dtype=param_dtype,
            reduce_dtype=reduce_dtype,
            buffer_dtype=buffer_dtype
        )

        # 参照モデルの場合、勾配計算をしないため、混合精度を無効化
        if self._is_ref:
            mixed_precision = None

        # アクターモジュールのFSDPラップ用の自動ラップポリシーを取得
        auto_wrap_policy = get_fsdp_wrap_policy(
            module=actor_module,
            config=fsdp_config.get('wrap_policy', None)
        )

        # HFRolloutを使用する場合
        if self._is_rollout and self.config.rollout.name == 'hf':
            # Gemma環境でハングする問題を回避するため、自動ラップポリシーを無効化
            auto_wrap_policy = None

        logger.debug(f'wrap_policy: {auto_wrap_policy}')

        # TODO(sgm): support hybrid
        # 自動ラップポリシーが指定されていない場合
        if auto_wrap_policy is None:

            # FSDPのシャーディング戦略をSHARD_GRAD_OPに設定
            sharding_strategy = ShardingStrategy.SHARD_GRAD_OP
        else:
            # FSDPのシャーディング戦略をFULL_SHARDに設定
            sharding_strategy = ShardingStrategy.FULL_SHARD

        # FSDPのパラメータ初期化関数を定義
        actor_module_fsdp = FSDP(
            actor_module,
            param_init_fn=init_fn,
            use_orig_params=False,
            auto_wrap_policy=auto_wrap_policy,
            device_id=torch.cuda.current_device(),
            sharding_strategy=sharding_strategy,  # zero3
            mixed_precision=mixed_precision,
            sync_module_states=True,
            device_mesh=self.device_mesh,
            forward_prefetch=False
        )

        # メモリ使用量をログに記録
        log_gpu_memory_usage('After Actor FSDP init', logger=logger)


        # アクターモデルの場合
        if self._is_actor:

            # AdamWオプティマイザを初期化
            actor_optimizer = optim.AdamW(
                actor_module_fsdp.parameters(),
                lr=optim_config.lr,
                betas=optim_config.get('betas', (0.9, 0.999)),
                weight_decay=optim_config.get('weight_decay', 1e-2)
            )

            # 最適化関数の設定から総トレーニングステップ数を取得
            total_steps = optim_config.get('total_training_steps', 0)
            logger.info(f"全てのトレーニングステップ数: {total_steps}")

            # 最適化関数の設定からウォームアップステップ数の比率を取得
            num_warmup_steps_ratio = optim_config.get('lr_warmup_steps_ratio', 0.)
            logger.info(f"ウォームアップステップ数の比率: {num_warmup_steps_ratio}")

            # ウォームアップステップ数を計算
            num_warmup_steps = int(num_warmup_steps_ratio * total_steps)
            logger.info(f"ウォームアップステップ数: {num_warmup_steps}")

            # 定数スケジューラを使用して学習率スケジューラを初期化
            actor_lr_scheduler = get_constant_schedule_with_warmup(
                optimizer=actor_optimizer,
                num_warmup_steps=num_warmup_steps
            )

        # アクターモデルでない場合、勾配計算を行わない
        else:
            actor_optimizer = None
            actor_lr_scheduler = None

        # メモリ使用量をログに記録
        log_gpu_memory_usage('After actor optimizer init', logger=logger)

        # 構築したモデル、オプティマイザ、学習率スケジューラ、モデル設定を返す
        return actor_module_fsdp, actor_optimizer, actor_lr_scheduler, actor_model_config

    def _build_rollout(self):
        """
        ロールアウトモジュールとシャーディングマネージャを構築する
        シャーディングマネージャはFSDPで分散された重みをvLLMに対応させるために使用する
        init_modelメソッドから呼び出される

        Returns:
            rollout (Rollout): ロールアウトモジュール
            rollout_sharding_manager (BaseShardingManager):
                ロールアウト用シャーディングマネージャ
        """
        # ロールアウト用のテンソルモデル並列サイズを取得
        infer_tp = self.config.rollout.tensor_model_parallel_size

        # データ並列サイズを計算
        dp = self.world_size // infer_tp

        # ロールアウト用のテンソルモデル並列サイズがワールドサイズで割り切れることを確認
        assert self.world_size % infer_tp == 0, f'rollout world_size: {self.world_size} is not divisible by infer_tp: {infer_tp}'

        # ロールアウト用のデバイスメッシュを初期化
        rollout_device_mesh = init_device_mesh(
            'cuda',
            mesh_shape=(dp, infer_tp),
            mesh_dim_names=['dp', 'infer_tp']
        )

        # HFロールアウトの場合
        # False
        if self.config.rollout.name == 'hf':

            # HFロールアウトを初期化
            rollout = HFRollout(
                module=self.actor_module_fsdp,
                config=self.config.rollout
            )

            # ベースクラスのシャーディングマネージャを初期化
            rollout_sharding_manager = BaseShardingManager()

        # vLLMロールアウトの場合
        # True
        elif self.config.rollout.name == 'vllm':

            # メモリ使用量をログに記録
            log_gpu_memory_usage('Before building vllm rollout', logger=None)

            # vLLMロールアウトを初期化
            rollout = vLLMRollout(
                actor_module=self.actor_module_fsdp,
                config=self.config.rollout,
                tokenizer=self.tokenizer,
                model_hf_config=self.actor_model_config
            )

            # メモリ使用量をログに記録
            log_gpu_memory_usage('After building vllm rollout', logger=None)

            # デバッグ目的でワールドサイズが1の場合、ロードフォーマットを'dummy_hf'に設定
            if torch.distributed.get_world_size() == 1:
                self.config.rollout.load_format = 'dummy_hf'

            # vLLM用のFSDPシャーディングマネージャを初期化
            rollout_sharding_manager = FSDPVLLMShardingManager(
                module=self.actor_module_fsdp,
                inference_engine=rollout.inference_engine,
                model_config=self.actor_model_config,
                full_params='hf' in self.config.rollout.load_format,
                device_mesh=rollout_device_mesh
            )

            # メモリ使用量をログに記録
            log_gpu_memory_usage('After building sharding manager', logger=None)

        # 構築したロールアウトモジュールとシャーディングマネージャを返す
        return rollout, rollout_sharding_manager

    @register(dispatch_mode=Dispatch.ONE_TO_ALL)
    def init_model(self):
        """
        モデル、オプティマイザ、ロールアウトモジュールを初期化する
        ワーカーの役割に応じて適切なコンポーネントを初期化する
        """
        logger.info("モデル初期化を開始")

        # 設定で指定された外部ライブラリをインポート
        # None
        import_external_libs(self.config.model.get('external_lib', None))

        # 上書き用のモデル設定を取得
        # {}
        override_model_config = OmegaConf.to_container(
            self.config.model.get('override_config', OmegaConf.create())
        )

        # パディング除去を使用するかどうかを取得
        # False
        use_remove_padding = self.config.model.get('use_remove_padding', False)

        # アクターまたはロールアウトの場合
        if self._is_actor or self._is_rollout:

            # アクターモデルの場合
            if self._is_actor:

                # 最適化関数の設定を取得
                # {'lr': 1e-06, 'lr_warmup_steps_ratio': 0.0, 'min_lr_ratio': None, 'warmup_style': 'constant', 'total_training_steps': -1},
                optim_config = self.config.actor.optim

                # FSDPの設定を取得
                # {'wrap_policy': {'min_num_params': 0}, 'param_offload': False, 'grad_offload': False, 'optimizer_offload': False, 'fsdp_size': -1}}
                fsdp_config = self.config.actor.fsdp_config

            # ロールアウトのみの場合
            else:

                # 最適化関数の設定はNone
                optim_config = None

                # FSDPの設定を空にする
                fsdp_config = OmegaConf.create()

            # アクターのモデル、最適化関数、学習率スケジューラ、設定を構築
            self.actor_module_fsdp, \
            self.actor_optimizer, \
            self.actor_lr_scheduler, \
            self.actor_model_config = \
                self._build_model_optimizer(
                model_path=self.config.model.path,
                fsdp_config=fsdp_config,
                optim_config=optim_config,
                override_model_config=override_model_config,
                use_remove_padding=use_remove_padding,
                enable_gradient_checkpointing=
                    self.config.model.get('enable_gradient_checkpointing', False),
                trust_remote_code=self.config.model.get('trust_remote_code', False)
            )

            # ラップされていない元のモジュールを取得
            self.actor_module = self.actor_module_fsdp._fsdp_wrapped_module

            # パラメータをオフロードする場合
            if self._is_offload_param:

                # パラメータをオフロード
                offload_fsdp_grad(module=self.actor_module_fsdp)

                # メモリ使用量をログに記録
                log_gpu_memory_usage(
                    'After offload actor grad during init', logger=logger
                )

            # オプティマイザをオフロードする場合
            if self._is_offload_optimizer:

                # オプティマイザをオフロード
                offload_fsdp_optimizer(optimizer=self.actor_optimizer)

                # メモリ使用量をログに記録
                log_gpu_memory_usage(
                    'After offload actor optimizer during init', logger=logger
                )

        # アクターモデルの場合
        if self._is_actor:

            # アクター設定の構造を固定
            OmegaConf.set_struct(self.config.actor, True)

            # パディング除去の使用設定を保存
            with open_dict(self.config.actor):
                self.config.actor.use_remove_padding = use_remove_padding

            # PPOアクターを初期化
            self.actor = DataParallelPPOActor(
                config=self.config.actor,
                actor_module=self.actor_module_fsdp,
                actor_optimizer=self.actor_optimizer
            )

        # ロールアウトモジュールの場合
        if self._is_rollout:

            # ロールアウトモジュールとシャーディングマネージャを構築
            self.rollout, self.rollout_sharding_manager = self._build_rollout()

        # 参照モデルの場合
        if self._is_ref:

            # 参照モデルのFSDPモジュールを構築
            self.ref_module_fsdp = self._build_model_optimizer(
                model_path=self.config.model.path,
                fsdp_config=self.config.ref.fsdp_config,
                optim_config=None,
                override_model_config=override_model_config,
                use_remove_padding=use_remove_padding,
                trust_remote_code=self.config.model.get(
                    'trust_remote_code', False)
                )[0]

            # パラメータをオフロードする場合
            if self._is_offload_param:

                # パラメータと勾配をオフロード
                offload_fsdp_param_and_grad(
                    module=self.ref_module_fsdp,
                    offload_grad=self._is_offload_grad
                )

            # 参照モデル設定の構造を固定
            OmegaConf.set_struct(self.config.ref, True)

            # パディング除去の使用設定を保存
            with open_dict(self.config.ref):
                self.config.ref.use_remove_padding = use_remove_padding

            # 参照モデルを初期化
            self.ref_policy = DataParallelPPOActor(
                config=self.config.ref,
                actor_module=self.ref_module_fsdp
            )

        # アクターモデルの場合、FLOPSカウンタを初期化
        # FLOPSカウンタはモデルの計算量を測定するためのモジュール
        if self._is_actor:
            self.flops_counter = FlopsCounter(self.actor_model_config)

        # 不要なGPUメモリを解放
        torch.cuda.empty_cache()

    @register(dispatch_mode=Dispatch.DP_COMPUTE_PROTO)
    def update_actor(self, data: DataProto):
        """
        アクターモデルを更新する

        Args:
            data (DataProto): トレーニングデータ
        Returns:
            output (DataProto): トレーニングメトリクスを含むデータプロトコル
        """

        # データをGPUに転送
        data = data.to('cuda')

        # アクターモデルであることを確認
        assert self._is_actor

        # パラメータをオフロードしている場合
        if self._is_offload_param:

            # パラメータと勾配をロード
            load_fsdp_param_and_grad(
                module=self.actor_module_fsdp,
                device_id=torch.cuda.current_device(),
                load_grad=self._is_offload_grad
            )

        # 最適化関数をオフロードしている場合
        if self._is_offload_optimizer:

            # 最適化関数をロード
            load_fsdp_optimizer(
                optimizer=self.actor_optimizer,
                device_id=torch.cuda.current_device()
            )

        # バッチデータをGPUに転送
        data.batch = data.batch.cuda()

        # メモリ使用量をログに記録
        log_gpu_memory_usage('Before update policy', logger=logger)

        # Ulyssesシャーディングマネージャのコンテキストで処理
        with self.ulysses_sharding_manager:

            # データを前処理
            data = self.ulysses_sharding_manager.preprocess_data(data=data)

            # タイマーを開始して、ポリシーを更新
            with Timer(name='update_policy', logger=None) as timer:
                metrics = self.actor.update_policy(data=data)

            # 経過時間を取得
            delta_time = timer.last

            # グローバルトークン数を取得
            global_num_tokens = data.meta_info['global_token_num']

            # FLOPSを計算
            estimated_flops, promised_flops = \
                self.flops_counter.estimate_flops(global_num_tokens, delta_time)

            # MFU（Model FLOPS Utilization）を計算してメトリクスに追加
            metrics['mfu/actor'] = \
                estimated_flops * self.config.actor.ppo_epochs \
                / promised_flops / self.world_size

            # 学習率スケジューラをステップ
            self.actor_lr_scheduler.step()

            # 現在の学習率を取得
            lr = self.actor_lr_scheduler.get_last_lr()[0]

            # 学習率をメトリクスに追加
            metrics['actor/lr'] = lr

            # メモリ使用量をログに記録
            log_gpu_memory_usage('After update policy', logger=logger)

            # トレーニングメトリクスを含むデータプロトコルを作成
            output = DataProto(meta_info={'metrics': metrics})

            # データを後処理
            output = self.ulysses_sharding_manager.postprocess_data(data=output)

            # データをCPUに転送
            output = output.to('cpu')

        # パラメータをオフロードする場合
        if self._is_offload_param:

            # パラメータと勾配をオフロード
            offload_fsdp_param_and_grad(
                module=self.actor_module_fsdp,
                offload_grad=self._is_offload_grad
            )

        # 最適化関数をオフロードする場合
        if self._is_offload_optimizer:

            # 最適化関数をオフロード
            offload_fsdp_optimizer(optimizer=self.actor_optimizer)

        # 不要なGPUメモリを解放
        torch.cuda.empty_cache()

        return output

    @register(dispatch_mode=Dispatch.DP_COMPUTE_PROTO)
    def generate_sequences(self, prompts: DataProto):
        """
        ロールアウトモジュールを使用してシーケンスを生成する

        Args:
            prompts (DataProto): プロンプトデータ
        Returns:
            output (DataProto): 生成されたシーケンスデータ
        """

         # データをGPUに転送
        prompts = prompts.to('cuda')

        # 対数確率を再計算するかどうかを取得
        # True
        recompute_log_prob = prompts.meta_info.get('recompute_log_prob', True)

        # ロールアウトモジュールであることを確認
        assert self._is_rollout

        # パラメータをオフロードしている場合
        if self._is_offload_param:

            # パラメータと勾配をロード
            load_fsdp_param_and_grad(
                module=self.actor_module_fsdp,
                device_id=torch.cuda.current_device(),
                load_grad=self._is_offload_grad
            )

        # バッチデータをGPUに転送
        prompts.batch = prompts.batch.cuda()

        # EOSトークンIDとパッドトークンIDをメタ情報に追加
        meta_info = {
            'eos_token_id': self.tokenizer.eos_token_id,
            'pad_token_id': self.tokenizer.pad_token_id
        }
        prompts.meta_info.update(meta_info)


        # ロールアウトシャーディングマネージャのコンテキストで処理
        with self.rollout_sharding_manager:

            # メモリ使用量をログに記録
            log_gpu_memory_usage(
                'After entering rollout sharding manager', logger=logger
            )

            # データを前処理
            prompts = self.rollout_sharding_manager.preprocess_data(prompts)

            # シーケンスを生成
            output = self.rollout.generate_sequences(prompts=prompts)

            # メモリ使用量をログに記録
            log_gpu_memory_usage('After rollout generation', logger=logger)

            # データを後処理 
            output = self.rollout_sharding_manager.postprocess_data(output)

        # アクターモデルでかつ、対数確率を再計算する場合
        if self._is_actor and recompute_log_prob:

            # 出力のメタ情報にマイクロバッチサイズを追加
            output.meta_info['micro_batch_size'] = \
                self.config.rollout.log_prob_micro_batch_size

            # 出力のメタ情報に最大トークン長を追加
            output.meta_info['max_token_len'] = \
                self.config.rollout.log_prob_max_token_len_per_gpu

            # 出力のメタ情報に動的バッチサイズ使用フラグを追加
            output.meta_info['use_dynamic_bsz'] = \
                self.config.rollout.log_prob_use_dynamic_bsz

            # 出力のメタ情報に温度パラメータを追加
            output.meta_info['temperature'] = \
                self.config.rollout.temperature

            # Ulyssesシャーディングマネージャのコンテキストで処理
            with self.ulysses_sharding_manager:

                # データを前処理
                output = self.ulysses_sharding_manager.preprocess_data(output)

                # 対数確率を計算
                old_log_probs = self.actor.compute_log_prob(data=output)

                # 対数確率を出力データに追加
                output.batch['old_log_probs'] = old_log_probs

                # データを後処理
                output = self.ulysses_sharding_manager.postprocess_data(output)

        # データをCPUに転送
        output = output.to('cpu')

        # パラメータをオフロードする場合
        if self._is_offload_param:

            # パラメータと勾配をオフロード
            offload_fsdp_param_and_grad(
                module=self.actor_module_fsdp,
                offload_grad=self._is_offload_grad
            )

        # KVキャッシュをクリア
        torch.cuda.empty_cache()

        # メモリ使用量をログに記録
        log_gpu_memory_usage('After recompute log prob', logger=logger)

        return output

    @register(dispatch_mode=Dispatch.DP_COMPUTE_PROTO)
    def compute_ref_log_prob(self, data: DataProto):
        """
        参照モデルを使用して対数確率を計算する

        Args:
            data (DataProto): 入力データ
        Returns:
            output (DataProto): 対数確率を含むデータプロトコル
        """

        # 参照モデルであることを確認
        assert self._is_ref

        # データをGPUに転送
        data = data.to('cuda')

        # パラメータをオフロードしている場合
        if self._is_offload_param:

            # パラメータと勾配をロード
            load_fsdp_param_and_grad(
                module=self.ref_module_fsdp,
                device_id=torch.cuda.current_device(),
                load_grad=self._is_offload_grad
            )

        # マイクロバッチサイズを取得
        micro_batch_size = self.config.ref.log_prob_micro_batch_size

        # データのメタ情報にマイクロバッチサイズを追加
        data.meta_info['micro_batch_size'] = micro_batch_size

        # データのメタ情報に温度パラメータを追加
        data.meta_info['temperature'] = self.config.rollout.temperature

        # データのメタ情報に最大トークン長を追加
        data.meta_info['max_token_len'] = \
            self.config.ref.log_prob_max_token_len_per_gpu

        # データのメタ情報に動的バッチサイズ使用フラグを追加
        data.meta_info['use_dynamic_bsz'] = \
            self.config.ref.log_prob_use_dynamic_bsz

        # Ulysses通信を開始（長いシーケンスを並列処理）
        with self.ulysses_sharding_manager:

            # 入力データを分割する
            data = self.ulysses_sharding_manager.preprocess_data(data)

            # 参照モデルで入力データを並列処理して、トークンの対数確率を計算
            output = self.ref_policy.compute_log_prob(data=data)

            # 対数確率をデータプロトコルに格納
            output = DataProto.from_dict(tensors={'ref_log_prob': output})

            # 分割された対数確率データを集約する
            output = self.ulysses_sharding_manager.postprocess_data(output)

        # データをCPUに転送
        output = output.to('cpu')

        # パラメータをオフロードする場合
        if self._is_offload_param:

            # パラメータと勾配をオフロード
            offload_fsdp_param_and_grad(
                module=self.ref_module_fsdp,
                offload_grad=self._is_offload_grad
            )

        # 不要なGPUメモリを解放
        torch.cuda.empty_cache()

        return output

    @register(dispatch_mode=Dispatch.ONE_TO_ALL)
    def save_checkpoint(self, local_path, hdfs_path=None):
        """
        アクターモデルのチェックポイントを保存する
        HDFSパスが指定されている場合、HDFSにもアップロードする

        Args:
            local_path (str): ローカルの保存パス
            hdfs_path (str, optional): HDFSの保存パス
        """

        # アクターモデルであることを確認
        assert self._is_actor

        # パラメータをオフロードしている場合
        if self._is_offload_param:

            # パラメータと勾配をロード
            load_fsdp_param_and_grad(
                module=self.actor_module_fsdp,
                device_id=torch.cuda.current_device(),
                load_grad=self._is_offload_grad
            )

        # stete dict（モデルや最適化関数の状態）の集約方法を定義
        cfg = FullStateDictConfig(
            offload_to_cpu=True, # CPUにオフロード
            rank0_only=True, # メインプロセスのみ
        )

        # state dictを集約する通信を開始
        with FSDP.state_dict_type(
            self.actor.actor_module,
            StateDictType.FULL_STATE_DICT,
            cfg
        ):
            # state dictを集約
            state_dict = self.actor.actor_module.state_dict()

        # メインプロセスの場合
        if self.rank == 0:
            logger.info(f'Saving actor checkpoint to {local_path}')

            # ローカルパスのディレクトリを作成
            os.makedirs(local_path, exist_ok=True)

            # アクターモジュールを保存
            self.actor_module.save_pretrained(local_path, state_dict=state_dict)

            # トークナイザを保存
            self.tokenizer.save_pretrained(local_path)

            # HDFSパスが指定されている場合
            if hdfs_path is not None:
                logger.info(f'Uploading actor checkpoint to {hdfs_path}')

                # HDFSのパスのディレクトリを作成
                hdfs_io.makedirs(hdfs_path, exist_ok=True)

                # ローカルからHDFSにチェックポイントをコピー
                hdfs_io.copy(src=local_path, dst=hdfs_path)

        # 全プロセスで同期
        torch.distributed.barrier()

        # パラメータをオフロードする場合
        if self._is_offload_param:

            # パラメータと勾配をオフロード
            offload_fsdp_param_and_grad(
                module=self.actor_module_fsdp,
                offload_grad=self._is_offload_grad
            )

#### CriticWorker

In [ ]:
class CriticWorker(Worker):
    """
    価値関数（クリティック）の学習・推論を担当するワーカークラス
    """

    def __init__(self, config):
        """
        Args:
            config (DictConfig): ワーカーの設定
        """
        logger.info(f"CriticWorkerを初期化 {config=}")

        super().__init__()

        # 分散処理が初期化されていない場合
        if not torch.distributed.is_initialized():

            # NCCLバックエンドで分散処理を初期化
            torch.distributed.init_process_group(backend="nccl")

        # 設定を属性に保存
        self.config = config

        # 1) ユリシーズシーケンス並列化のためにデバイスメッシュを構築

        # 全てのGPUの数を取得
        world_size = torch.distributed.get_world_size()

        # デバイスメッシュを初期化
        self.ulysses_device_mesh = None

        # ユリシーズシーケンス並列サイズを設定（デフォルトは1）
        self.ulysses_sequence_parallel_size = \
            self.config.get('ulysses_sequence_parallel_size', 1)

        # データ並列サイズ（data parallel）を計算
        dp = world_size // self.ulysses_sequence_parallel_size

        # ユリシーズシーケンス並列サイズが1より大きい場合
        if self.ulysses_sequence_parallel_size > 1:

            # デバイスメッシュを初期化
            # 形状は (データ並列サイズ, シーケンス並列サイズ)
            # データ並列は、バッチごとに異なるGPUにデータを分割して処理する手法
            # シーケンス並列は、長いシーケンスを複数のGPUに分割して処理する手法
            self.ulysses_device_mesh = init_device_mesh(
                'cuda',
                mesh_shape=(dp, self.ulysses_sequence_parallel_size),
                mesh_dim_names=['dp', 'sp']
            )

        # ユリシーズシャーディングマネージャを初期化
        # シーケンスの分割・集約などの通信や管理を行うモジュール
        self.ulysses_sharding_manager = FSDPUlyssesShardingManager(
            self.ulysses_device_mesh
        )

        # パラメータオフロードの設定を取得
        self._is_offload_param = self.config.model.fsdp_config.param_offload

        # 勾配オフロードの設定を取得
        self._is_offload_grad = self.config.model.fsdp_config.grad_offload

        # 最適化関数オフロードの設定を取得
        self._is_offload_optimizer = self.config.model.fsdp_config.optimizer_offload

        # ミニバッチサイズを計算
        self.config.ppo_mini_batch_size //= (
            torch.distributed.get_world_size() // self.ulysses_sequence_parallel_size
        )

        # マイクロバッチサイズを計算
        self.config.ppo_micro_batch_size //= (
            torch.distributed.get_world_size() // self.ulysses_sequence_parallel_size
        )

        # 順伝搬マイクロバッチサイズを計算
        self.config.forward_micro_batch_size //= (
            torch.distributed.get_world_size() // self.ulysses_sequence_parallel_size
        )

    def _build_critic_model_optimizer(self, config):
        """
        クリティックモデル、オプティマイザ、学習率スケジューラを構築する

        Args:
            config (DictConfig): ワーカーの設定
        Returns:
            critic_module (FSDP): クリティックモデルのFSDPラップ
            critic_optimizer (Optimizer): クリティックモデルのオプティマイザ
            critic_lr_scheduler (LRScheduler): クリティックモデルの学習率スケジューラ
        """

        # コピー元のHDFSパスからローカルパスにモデルをコピー
        local_path = copy_local_path_from_hdfs(config.model.path)

        # HDFSパスからローカルパスにトークナイザをコピー
        tokenizer_path = copy_local_path_from_hdfs(config.model.tokenizer_path)

        # Hugging Faceのトークナイザを初期化
        self.tokenizer = hf_tokenizer(
            tokenizer_path,
            trust_remote_code=config.model.get('trust_remote_code', False)
        )

        # モデルの上書き設定を取得
        override_config = OmegaConf.to_container(
            self.config.model.get('override_config', OmegaConf.create())
        )

        # トークナイザの特殊トークンIDを上書き設定に追加
        override_config_kwargs = {
            'bos_token_id': self.tokenizer.bos_token_id,
            'eos_token_id': self.tokenizer.eos_token_id,
            'pad_token_id': self.tokenizer.pad_token_id,
        }

        # 上書き設定をマージ
        override_config_kwargs.update(override_config)

        # メインプロセスの場合
        if self.rank == 0:
            print(f'クリティックの上書き設定: {override_config_kwargs}')

        # モデルのデータ型を取得
        torch_dtype = self.config.model.fsdp_config.get('model_dtype', 'fp32')

        # データ型をPyTorchのdtypeに変換
        torch_dtype = PrecisionType.to_dtype(torch_dtype)

        trust_remote_code = False

        # モデル設定を初期化
        critic_model_config = AutoConfig.from_pretrained(
            local_path,
            trust_remote_code=trust_remote_code
        )

        # クリティックモデルのラベル数を1に設定
        critic_model_config.num_labels = 1

        # パディング除去を使用するかどうかを取得
        use_remove_padding = config.model.get('use_remove_padding', False)

        # パディング除去を使用する場合、モデルが対応しているか確認
        if use_remove_padding:
            check_model_support_rmpad(critic_model_config.model_type)

        # パディング除去を使用し、かつシーケンス並列サイズが1より大きい場合
        if use_remove_padding and self.ulysses_sequence_parallel_size > 1:
            # モンキーパッチを適用して、パディング除去を有効化
            apply_monkey_patch(critic_model_config, verbose=True)

        # モデルのパラメータを初期化するコンテキストマネージャを取得
        init_context = get_init_weight_context_manager()

        with init_context(), warnings.catch_warnings():

            # 警告を無視
            warnings.simplefilter("ignore")

            # ドロップアウトを無効化
            setattr(critic_model_config, 'classifier_dropout', 0.)
            setattr(critic_model_config, 'hidden_dropout', '0')

            # クリティックモデルを初期化
            critic_module = \
                AutoModelForTokenClassification.from_pretrained(
                    pretrained_model_name_or_path=local_path,
                    torch_dtype=torch_dtype,
                    config=critic_model_config,
                    attn_implementation='flash_attention_2',
                    trust_remote_code=trust_remote_code
                )

            # クリティックモデルを指定のデータ型に変換
            critic_module.to(torch_dtype)

            # 勾配チェックポイントを有効化する場合
            if config.model.get('enable_gradient_checkpointing', False):

                # 勾配チェックポイントを有効化
                critic_module.gradient_checkpointing_enable(
                    gradient_checkpointing_kwargs={'use_reentrant': False}
                )

        # メインプロセスの場合
        if self.rank == 0:

            # クリティックモデルのサイズを表示
            print_model_size(critic_module)


        # クリティックモデルの設定を属性に保存
        self.critic_model_config = critic_model_config

        # FSDPの初期化関数を取得
        fsdp_config = self.config.model.fsdp_config

        # 自動混合精度の設定を取得
        mixed_precision_config = fsdp_config.get('mixed_precision', None)

        # 自動混合精度が設定されている場合
        if mixed_precision_config is not None:

            # パラメータのデータ型を取得
            param_dtype = PrecisionType.to_dtype(
                mixed_precision_config.get('param_dtype', 'bf16')
            )

            # 集約のデータ型を取得
            reduce_dtype = PrecisionType.to_dtype(
                mixed_precision_config.get('reduce_dtype', 'fp32')
            )

            # バッファのデータ型を取得
            buffer_dtype = PrecisionType.to_dtype(
                mixed_precision_config.get('buffer_dtype', 'fp32')
            )

        # 自動混合精度が設定されていない場合
        else:
            param_dtype = torch.bfloat16
            reduce_dtype = torch.float32
            buffer_dtype = torch.float32

        # FSDPの自動混合精度を初期化
        mixed_precision = MixedPrecision(
            param_dtype=param_dtype,
            reduce_dtype=reduce_dtype,
            buffer_dtype=buffer_dtype
        )

        # 自動ラップポリシーを取得
        auto_wrap_policy = get_fsdp_wrap_policy(
            module=critic_module,
            config=self.config.model.fsdp_config.wrap_policy
        )

        # GPUメモリ使用量をログに記録
        log_gpu_memory_usage('Before critic FSDP', logger=None)

        # クリティックモデルをFSDPでラップ
        critic_module = FSDP(
            critic_module,
            param_init_fn=init_fn,
            use_orig_params=False,
            auto_wrap_policy=auto_wrap_policy,
            device_id=torch.cuda.current_device(),
            sharding_strategy=ShardingStrategy.FULL_SHARD,
            mixed_precision=mixed_precision,
            sync_module_states=True,
            forward_prefetch=False
        )

        # GPUメモリ使用量をログに記録
        log_gpu_memory_usage('After critic FSDP', logger=None)

        # AdamWオプティマイザを初期化
        critic_optimizer = optim.AdamW(
            critic_module.parameters(),
            lr=config.optim.lr,
            betas=config.optim.get('betas', (0.9, 0.999)),
            weight_decay=config.optim.get('weight_decay', 1e-2)
        )

        # 総トレーニングステップ数を取得
        total_steps = config.optim.get('total_training_steps', 0)
        print(f"クリティックモデルの総トレーニングステップ数: {total_steps}")

        # 学習率のウォームアップステップ数の比率を取得
        num_warmup_steps_ratio = config.optim.get('lr_warmup_steps_ratio', 0.)

        # 学習率のウォームアップステップ数を計算
        num_warmup_steps = int(num_warmup_steps_ratio * total_steps)
        print(f"クリティックモデルの学習率ウォームアップステップ数: {num_warmup_steps}")

        # 定数学習率スケジューラを初期化
        critic_lr_scheduler = get_constant_schedule_with_warmup(
            optimizer=critic_optimizer,
            num_warmup_steps=num_warmup_steps
        )

        # クリティックモデル、オプティマイザ、学習率スケジューラを返す
        return critic_module, critic_optimizer, critic_lr_scheduler

    @register(dispatch_mode=Dispatch.ONE_TO_ALL)
    def init_model(self):
        """
        クリティックモデルを初期化する
        """

        # 外部ライブラリが指定されている場合、インポート
        import_external_libs(self.config.model.get('external_lib', None))

        # クリティックモデル、オプティマイザ、学習率スケジューラを構築
        self.critic_module, self.critic_optimizer, self.critic_lr_scheduler = \
            self._build_critic_model_optimizer(self.config)

        # パラメータをオフロードする場合
        if self._is_offload_param:

            # パラメータと勾配をオフロード
            offload_fsdp_param_and_grad(
                module=self.critic_module,
                offload_grad=self._is_offload_grad
            )

        # 最適化関数をオフロードする場合
        if self._is_offload_optimizer:

            # 最適化関数をオフロード
            offload_fsdp_optimizer(optimizer=self.critic_optimizer)

        # クリティックモデルを初期化
        self.critic = DataParallelPPOCritic(
            config=self.config,
            critic_module=self.critic_module,
            critic_optimizer=self.critic_optimizer
        )

        # クリティックモデルのFLOPSカウンタを初期化
        self.flops_counter = FlopsCounter(self.critic_model_config)

        # 不要なGPUメモリを解放
        torch.cuda.empty_cache()

    @register(dispatch_mode=Dispatch.DP_COMPUTE_PROTO)
    def compute_values(self, data: DataProto):
        """
        価値関数を計算する

        Args:
            data (DataProto): 入力データ
        Returns:
            output (DataProto): 価値関数の出力を含むデータプロトコル
        """

        # データをGPUに転送
        data = data.to('cuda')

        # パラメータをオフロードしている場合
        if self._is_offload_param:

            # パラメータと勾配をロード
            load_fsdp_param_and_grad(
                module=self.critic_module,
                device_id=torch.cuda.current_device(),
                load_grad=self._is_offload_grad
            )

        # マイクロバッチサイズを取得
        micro_batch_size = self.config.forward_micro_batch_size

        # マイクロバッチサイズをデータのメタ情報に追加
        data.meta_info['micro_batch_size'] = micro_batch_size

        # 最大トークン長をデータのメタ情報に追加
        data.meta_info['max_token_len'] = self.config.forward_max_token_len_per_gpu

        # 動的バッチサイズ使用フラグをデータのメタ情報に追加
        data.meta_info['use_dynamic_bsz'] = self.config.use_dynamic_bsz

        # ユリシーズ通信を開始（長いシーケンスを並列処理）
        with self.ulysses_sharding_manager:

            # 入力データを分割する
            data = self.ulysses_sharding_manager.preprocess_data(data=data)

            # 価値関数を計算
            values = self.critic.compute_values(data=data)

            # 価値関数の出力をデータプロトコルに格納
            output = DataProto.from_dict(tensors={'values': values})

            # 分割された価値関数データを集約する
            output = self.ulysses_sharding_manager.postprocess_data(data=output)

        # データをCPUに転送
        output = output.to('cpu')

        # パラメータをオフロードする場合
        if self._is_offload_param:

            # パラメータと勾配をオフロード
            offload_fsdp_param_and_grad(
                module=self.critic_module,
                offload_grad=self._is_offload_grad
            )

        # 不要なGPUメモリを解放
        torch.cuda.empty_cache()

        return output

    @register(dispatch_mode=Dispatch.DP_COMPUTE_PROTO)
    def update_critic(self, data: DataProto):
        """
        クリティックモデルを更新する

        Args:
            data (DataProto): トレーニングデータ
        Returns:
            output (DataProto): トレーニングメトリクスを含むデータプロトコル
        """

        # データをGPUに転送
        data = data.to('cuda')

        # パラメータをオフロードしている場合
        if self._is_offload_param:

            # パラメータと勾配をロード
            load_fsdp_param_and_grad(
                module=self.critic_module,
                device_id=torch.cuda.current_device(),
                load_grad=self._is_offload_grad
            )

        # 最適化関数をオフロードしている場合
        if self._is_offload_optimizer:

            # 最適化関数をロード
            load_fsdp_optimizer(
                optimizer=self.critic_optimizer,
                device_id=torch.cuda.current_device()
            )

        # ユリシーズ通信を開始（長いシーケンスを並列処理）
        with self.ulysses_sharding_manager:

            # 入力データを分割する
            data = self.ulysses_sharding_manager.preprocess_data(data=data)

            # タイマーを開始して、クリティックモデルを更新
            with Timer(name='update_critic', logger=None) as timer:
                metrics = self.critic.update_critic(data=data)

            # 経過時間を取得
            delta_time = timer.last

            # グローバルトークン数を取得
            global_num_tokens = data.meta_info['global_token_num']

            # FLOPSを計算
            estimated_flops, promised_flops = \
                self.flops_counter.estimate_flops(global_num_tokens, delta_time)

            # MFU（Model FLOPS Utilization）を計算してメトリクスに追加
            metrics['mfu/critic'] = \
                estimated_flops * self.config.ppo_epochs / \
                    promised_flops / self.world_size

            # 学習率スケジューラをステップ
            self.critic_lr_scheduler.step()

            # 現在の学習率を取得
            lr = self.critic_lr_scheduler.get_last_lr()[0]

            # 学習率をメトリクスに追加
            metrics['critic/lr'] = lr

            # トレーニングメトリクスを含むデータプロトコルを作成
            output = DataProto(batch=None, meta_info={'metrics': metrics})

            # 分割されたトレーニングメトリクスデータを集約する
            output = self.ulysses_sharding_manager.postprocess_data(data=output)

        # パラメータをオフロードする場合
        if self._is_offload_param:

            # パラメータと勾配をオフロード
            offload_fsdp_param_and_grad(
                module=self.critic_module,
                offload_grad=self._is_offload_grad
            )

        # 最適化関数をオフロードする場合
        if self._is_offload_optimizer:

            # 最適化関数をオフロード
            offload_fsdp_optimizer(optimizer=self.critic_optimizer)

        # 不要なGPUメモリを解放
        torch.cuda.empty_cache()

        # データをCPUに転送
        output = output.to('cpu')

        return output

    @register(dispatch_mode=Dispatch.ONE_TO_ALL)
    def save_checkpoint(self, local_path, hdfs_path=None):
        """
        クリティックモデルのチェックポイントを保存する

        Args:
            local_path (str): ローカルの保存パス
            hdfs_path (str, optional): HDFSの保存パス
        """

        # パラメータをオフロードしている場合
        if self._is_offload_param:

            # パラメータと勾配をロード
            load_fsdp_param_and_grad(
                module=self.critic_module,
                device_id=torch.cuda.current_device(),
                load_grad=self._is_offload_grad
            )

        # stete dict（モデルや最適化関数の状態）の集約方法を定義
        cfg = FullStateDictConfig(
            offload_to_cpu=True, # CPUにオフロード
            rank0_only=True # メインプロセスのみ
        )

        # state dictを集約する通信を開始
        with FSDP.state_dict_type(
            self.critic_module,
            StateDictType.FULL_STATE_DICT,
            cfg
        ):

            # state dictを集約
            state_dict = self.critic_module.state_dict()

        # メインプロセスの場合
        if self.rank == 0:
            print(f'クリティックのチェックポイントを保存: {local_path}')

            # ローカルパスのディレクトリを作成
            os.makedirs(local_path, exist_ok=True)

            # クリティックモジュールを保存
            self.critic_module._fsdp_wrapped_module.save_pretrained(
                local_path,
                state_dict=state_dict
            )

            # トークナイザを保存
            self.tokenizer.save_pretrained(local_path)

            # HDFSパスが指定されている場合
            if hdfs_path is not None:
                print(f'クリティックのチェックポイントをHDFSにアップロード: {hdfs_path}')

                # HDFSのパスのディレクトリを作成
                hdfs_io.makedirs(hdfs_path, exist_ok=True)

                # ローカルからHDFSにチェックポイントをコピー
                hdfs_io.copy(src=local_path, dst=hdfs_path)

        # 全プロセスで同期
        torch.distributed.barrier()

        # パラメータをオフロードする場合
        if self._is_offload_param:

            # パラメータと勾配をオフロード
            offload_fsdp_param_and_grad(
                module=self.critic_module,
                offload_grad=self._is_offload_grad
            )

#### RewardModelWorker

In [ ]:
class RewardModelWorker(Worker):
    """
    報酬モデルの推論を担当するワーカークラス
    今回は使わない
    """

    def __init__(self, config):
        super().__init__()
        print(f"RewardModelWorkerを初期化 {config=}")

        # 分散処理が初期化されていない場合
        if not torch.distributed.is_initialized():

            # NCCLバックエンドで分散処理を初期化
            torch.distributed.init_process_group(backend="nccl")

        # 設定を属性に保存
        self.config = config

        # ユリシーズシーケンス並列化のためにデバイスメッシュを構築

        # 全てのGPUの数を取得
        world_size = torch.distributed.get_world_size()

        # デバイスメッシュを初期化
        self.ulysses_device_mesh = None

        # ユリシーズシーケンス並列サイズを設定（デフォルトは1）
        self.ulysses_sequence_parallel_size = \
            self.config.get('ulysses_sequence_parallel_size', 1)

        # データ並列サイズ（data parallel）を計算
        dp = world_size // self.ulysses_sequence_parallel_size

        # ユリシーズシーケンス並列サイズが1より大きい場合
        if self.ulysses_sequence_parallel_size > 1:

            # デバイスメッシュを初期化
            self.ulysses_device_mesh = init_device_mesh(
                'cuda',
                mesh_shape=(dp, self.ulysses_sequence_parallel_size),
                mesh_dim_names=['dp', 'sp']
            )

        # ユリシーズシャーディングマネージャを初期化
        # シーケンスの分割・集約などの通信や管理を行うモジュール
        self.ulysses_sharding_manager = FSDPUlyssesShardingManager(
            self.ulysses_device_mesh
        )

        # パディング除去使用フラグを取得し、属性に保存
        self.use_remove_padding = self.config.model.get(
            'use_remove_padding', False
        )

        # プロセスごとのマイクロバッチサイズを計算
        self.config.micro_batch_size //= torch.distributed.get_world_size()

    def _build_model(self, config):
        """
        報酬モデルを構築する

        Args:
            config (DictConfig): ワーカーの設定
        Returns:
            reward_module (FSDP): FSDPでラップされた報酬モデル
        """

        # HDFSパスからローカルパスにモデルをコピー
        local_path = copy_local_path_from_hdfs(config.model.path)

        # 入力のトークナイザー（チャットテンプレート適応用）の設定がない場合
        if self.config.model.input_tokenizer is None:

            # チャットテンプレートを切り替えないフラグを無効化
            self._do_switch_chat_template = False

        # 入力のトークナイザー（チャットテンプレート適応用）の設定がある場合
        else:

            # チャットテンプレートを切り替えるフラグを有効化
            self._do_switch_chat_template = True

            # HDFSパスからローカルパスにトークナイザをコピー
            input_tokenizer_local_path = copy_local_path_from_hdfs(
                config.model.input_tokenizer
            )

            # Hugging Faceの入力のトークナイザを初期化
            self.input_tokenizer = hf_tokenizer(
                input_tokenizer_local_path,
                trust_remote_code=config.model.get('trust_remote_code', False)
            )

            # Hugging Faceのトークナイザを初期化 
            self.tokenizer = hf_tokenizer(
                local_path,
                trust_remote_code=config.model.get('trust_remote_code', False)
            )


        # trust_remote_code設定を取得
        trust_remote_code = config.model.get('trust_remote_code', False)

        # 学習済みのモデル設定を初期化
        model_config = AutoConfig.from_pretrained(
            local_path, trust_remote_code=trust_remote_code
        )

        # 報酬モデルのラベル数を1に設定（回帰タスク用）
        model_config.num_labels = 1

        # パディング除去を使用するかどうかを取得
        use_remove_padding = config.model.get('use_remove_padding', False)

        # パディング除去を使用する場合、モデルが対応しているか確認
        if use_remove_padding:
            check_model_support_rmpad(model_config.model_type)

        # パディング除去を使用し、かつシーケンス並列サイズが1より大きい場合
        if use_remove_padding and self.ulysses_sequence_parallel_size > 1:
            # モンキーパッチを適用して、パディング除去を有効化
            apply_monkey_patch(model_config, verbose=True)

        # note that we have to create model in fp32. Otherwise, the optimizer is in bf16, which is incorrect

        # モデルのパラメータを初期化するコンテキストマネージャを取得
        init_context = get_init_weight_context_manager(
            use_meta_tensor=not model_config.tie_word_embeddings
        )

        # 初期化コンテキストを開始
        with init_context(), warnings.catch_warnings():

            # 警告を無視
            warnings.simplefilter("ignore")

            # 分類器ドロップアウトを無効化
            setattr(model_config, 'classifier_dropout', 0.)

            # 報酬モデルを初期化
            reward_module = \
                AutoModelForTokenClassification.from_pretrained(
                    pretrained_model_name_or_path=local_path,
                    config=model_config,
                    torch_dtype=torch.bfloat16,
                    attn_implementation='flash_attention_2',
                    trust_remote_code=trust_remote_code
                )

            # 報酬モデルをbfloat16に変換
            reward_module.to(torch.bfloat16)

        # FSDPの自動ラップポリシーを取得
        auto_wrap_policy = get_fsdp_wrap_policy(
            module=reward_module,
            config=self.config.model.fsdp_config
        )

        # 報酬モデルをFSDPでラップ
        # これにより、モデルのパラメータが複数のGPUに分散され、メモリ使用量が削減される
        reward_module = FSDP(
            reward_module,
            param_init_fn=init_fn,
            use_orig_params=False,
            auto_wrap_policy=auto_wrap_policy,
            device_id=torch.cuda.current_device(),
            sharding_strategy=ShardingStrategy.FULL_SHARD, # zero3
            sync_module_states=True,
            cpu_offload=CPUOffload(
                offload_params=self.config.model.fsdp_config.param_offload
            ),
            forward_prefetch=False)

        return reward_module

    @register(dispatch_mode=Dispatch.ONE_TO_ALL)
    def init_model(self):
        """
        報酬モデルを初期化する
        """
        # 外部ライブラリが指定されている場合、インポート
        import_external_libs(self.config.model.get('external_lib', None))

        # 報酬モデルを構築
        self.reward_module = self._build_model(config=self.config)

        # 不要なGPUメモリを解放
        torch.cuda.empty_cache()

    def _forward_micro_batch(self, micro_batch):
        """
        マイクロバッチに対して順伝搬を行う

        Args:
            micro_batch (Dict): マイクロバッチのデータ
        Returns:
            rm_score (torch.Tensor): 報酬スコアのテンソル
        """

        # 勾配計算を無効化し、bfloat16で自動キャスト
        with torch.no_grad(), torch.autocast(
            device_type='cuda', dtype=torch.bfloat16
        ):

            # マイクロバッチの入力シーケンスIDを取得
            input_ids = micro_batch['input_ids']

            # バッチサイズとシーケンス長を取得
            batch_size, seqlen = input_ids.shape

            # マイクロバッチのアテンションマスクを取得
            attention_mask = micro_batch['attention_mask']

            # マイクロバッチの位置IDを取得
            position_ids = micro_batch['position_ids']

            # パディング除去を使用する場合
            if self.use_remove_padding:

                # パディングを除去して入力IDを整形
                input_ids_rmpad, indices, *_ = unpad_input(
                    input_ids.unsqueeze(-1),
                    attention_mask
                )

                # 転置
                # (seq_len, batch_size) -> (batch_size, seq_len) 
                # (1, total_nnz)
                input_ids_rmpad = input_ids_rmpad.transpose(0, 1)

                # 回転位置エンコーディングを適用するために位置IDをパディング除去
                position_ids_rmpad = index_first_axis(
                    rearrange(position_ids.unsqueeze(-1), "b s ... -> (b s) ..."),
                    indices
                ).transpose(0, 1)

                # ユリシーズシーケンス並列サイズが1より大きい場合
                if self.ulysses_sequence_parallel_size > 1:

                    # 入力IDと位置IDをパディングしてスライス
                    input_ids_rmpad, position_ids_rmpad, pad_size = \
                        ulysses_pad_and_slice_inputs(
                            input_ids_rmpad, 
                            sp_size=self.ulysses_sequence_parallel_size
                        )

                # Flash Attention Varlenを使用して報酬モデルを順伝搬
                output = self.reward_module(
                    input_ids=input_ids_rmpad,
                    attention_mask=None,
                    position_ids=position_ids_rmpad,
                    use_cache=False
                )

                # 報酬モデルの出力ロジットを取得
                reward_rmpad = output.logits

                # 余分な次元を削除
                # (total_nnz)
                reward_rmpad = reward_rmpad.squeeze(0)

                # シーケンス並列サイズが1より大きい場合
                if self.ulysses_sequence_parallel_size > 1:

                    # 出力を集約
                    reward_rmpad = gather_outpus_and_unpad(
                        reward_rmpad,
                        gather_dim=0,
                        unpad_dim=0,
                        padding_size=pad_size
                    )

                # パディングを元に戻して報酬スコアを取得
                rm_score = pad_input(
                    reward_rmpad,
                    indices=indices,
                    batch=batch_size,
                    seqlen=seqlen
                ).squeeze(-1)

            # パディング除去を使用しない場合
            else:

                # Flash Attentionを使用して報酬モデルを順伝搬
                output = self.reward_module(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    position_ids=position_ids
                )

                # 報酬モデルの出力ロジットを取得
                # (batch_size, seq_len, 1)
                rm_score = output.logits

                # 余分な次元を削除
                rm_score = rm_score.squeeze(-1)

            # 最後の有効なトークンの位置を取得
            # (bsz,)
            eos_mask_idx = torch.argmax(position_ids * attention_mask, dim=-1)

            # バッチごとに最後の有効なトークンの報酬スコアを取得
            rm_score = rm_score[torch.arange(batch_size), eos_mask_idx]

            return rm_score

    def _expand_to_token_level(self, data: DataProto, scores: torch.Tensor):
        """
        スコアをトークンレベルに拡張する

        Args:
            data (DataProto): 入力データ
            scores (torch.Tensor): 報酬スコアのテンソル
        Returns:
            token_level_scores (torch.Tensor): トークンレベルの報酬スコアのテンソル
        """

        # バッチサイズを取得
        batch_size = data.batch.batch_size[0]

        # expand as token_level_reward

        # マイクロバッチのアテンションマスクを取得
        attention_mask = data.batch['attention_mask']

        # マイクロバッチの位置IDを取得
        position_ids = data.batch['position_ids']

        # 応答の長さを取得
        response_length = data.batch['responses'].shape[-1]

        # 最後の有効なトークンの位置を取得
        # (bsz,)
        eos_mask_idx = torch.argmax(position_ids * attention_mask, dim=-1)

        # トークンレベルのスコアを初期化
        # (bsz, seqlen)
        token_level_scores = torch.zeros_like(attention_mask, dtype=scores.dtype)

        # バッチごとに最後の有効なトークンのスコアを設定
        token_level_scores[torch.arange(batch_size), eos_mask_idx] = scores

        # 応答部分にスコアを拡張 
        token_level_scores = token_level_scores[:, -response_length:]

        return token_level_scores

    def _switch_chat_template(self, data: DataProto):
        """
        チャットテンプレートを切り替える

        Args:
            data (DataProto): 入力データ
        Returns:
            rm_data (DataProto): チャットテンプレートが切り替えられたデータ
        """

        # マイクロバッチの最大シーケンス長を取得
        src_max_length = data.batch['attention_mask'].shape[-1]

        src_tokenizer = self.input_tokenizer
        target_tokenizer = self.tokenizer

        rm_input_ids = []
        rm_attention_mask = []

        for i in range(data.batch.batch_size[0]):
            # extract raw prompt
            chat: list = data.non_tensor_batch['raw_prompt'][i].tolist()

            # extract response
            response_ids = data.batch['responses'][i]
            response_length = response_ids.shape[-1]
            valid_response_length = data.batch['attention_mask'][i][-response_length:].sum()
            valid_response_ids = response_ids[:valid_response_length]

            # decode
            response = src_tokenizer.decode(valid_response_ids)
            # remove bos and eos
            response = response.replace(src_tokenizer.eos_token, '')

            chat.append({'role': 'assistant', 'content': response})

            prompt_with_chat_template = target_tokenizer.apply_chat_template(
                chat,
                add_generation_prompt=False,
                tokenize=False
            )

            if self.rank == 0 and i == 0:
                # for debugging purpose
                print(f'Switch template. chat: {prompt_with_chat_template}')

            # the maximum length is actually determined by the reward model itself
            max_length = self.config.get('max_length', src_max_length)
            if max_length is None:
                max_length = src_max_length
            input_ids, attention_mask = verl_F.tokenize_and_postprocess_data(
                prompt=prompt_with_chat_template,
                tokenizer=target_tokenizer,
                max_length=max_length,
                pad_token_id=target_tokenizer.pad_token_id,
                left_pad=False,  # right padding
                truncation=self.config.get('truncation', 'right'))  # truncate from the right

            rm_input_ids.append(input_ids)
            rm_attention_mask.append(attention_mask)

        rm_input_ids = torch.cat(rm_input_ids, dim=0)
        rm_attention_mask = torch.cat(rm_attention_mask, dim=0)

        rm_position_ids = compute_position_id_with_mask(rm_attention_mask)

        rm_inputs = {'input_ids': rm_input_ids, 'attention_mask': rm_attention_mask, 'position_ids': rm_position_ids}

        return DataProto.from_dict(rm_inputs)

    @register(dispatch_mode=Dispatch.DP_COMPUTE_PROTO)
    def compute_rm_score(self, data: DataProto):
        data = data.to('cuda')
        if self._do_switch_chat_template:
            rm_data = self._switch_chat_template(data)

        rm_data.batch = rm_data.batch.cuda()

        # perform forward computation
        with self.ulysses_sharding_manager:
            rm_data = self.ulysses_sharding_manager.preprocess_data(data=rm_data)
            data = self.ulysses_sharding_manager.preprocess_data(data=data)

            use_dynamic_bsz = self.config.use_dynamic_bsz
            if use_dynamic_bsz:
                max_token_len = self.config.forward_max_token_len_per_gpu * self.ulysses_sequence_parallel_size
                micro_batches, indices = rearrange_micro_batches(batch=rm_data.batch, max_token_len=max_token_len)
            else:
                micro_batches = rm_data.batch.split(self.config.micro_batch_size)
            output = []
            for micro_batch in micro_batches:
                rm_score = self._forward_micro_batch(micro_batch)
                output.append(rm_score)
            scores = torch.cat(output, dim=0)  # (batch_size)

            if use_dynamic_bsz:
                indices = list(itertools.chain.from_iterable(indices))
                assert len(indices) == scores.size(0), f"{len(indices)} vs. {scores.size()}"
                revert_indices = torch.tensor(get_reverse_idx(indices), dtype=torch.long)
                scores = scores[revert_indices]

            token_level_scores = self._expand_to_token_level(data, scores)
            # Note that this is only the scores, may not be the final rewards used to train RL
            output = DataProto.from_dict(tensors={'rm_scores': token_level_scores})
            output = self.ulysses_sharding_manager.postprocess_data(data=output)

        output = output.to('cpu')
        torch.cuda.empty_cache()
        return output


### ワーカーグループ

In [ ]:
def get_random_string(length: int) -> str:
    """
    指定された長さのランダムな英数字の文字列を生成する
    RayWokerGroupでユニークなIDを生成するために使用

    Args:
        length (int): 生成する文字列の長さ
    Returns:
        str: 生成されたランダムな文字列
    """
    letters_digits = string.ascii_letters + string.digits
    return ''.join(random.choice(letters_digits) for _ in range(length))

In [ ]:
def func_generator(self, method_name, dispatch_fn, collect_fn, execute_fn, blocking):
    """
    Rayのリモートメソッドを生成する関数
    リモートメソッドは、引数を分散処理用に変換し、指定されたメソッドを実行し、結果を集約する
    複雑な分散処理を抽象化

    Args:
        self: ワーカーインスタンス
        method_name (str): 呼び出すメソッドの名前
        dispatch_fn (callable): 引数を分散処理用に変換する関数
        collect_fn (callable): 結果を集約する関数
        execute_fn (callable): メソッドを実行する関数
        blocking (bool): ブロッキングモードで実行するかどうかのフラグ
    Returns:
        func (callable): 生成されたリモートメソッド
    """

    def func(*args, **kwargs):
        # 引数を分散処理用に変換
        args, kwargs = dispatch_fn(self, *args, **kwargs)

        # 指定されたメソッドを実行
        output = execute_fn(method_name, *args, **kwargs)

        # 同期実行の場合
        if blocking:
            # 結果を取得
            output = ray.get(output)

        # 結果を集約
        output = collect_fn(self, output)

        return output

    return func

In [ ]:
class RayClassWithInitArgs(ClassWithInitArgs):
    """
    Rayのリモートワーカーを生成するためのファクトリー
    RayWorkerGroupで使用される
    """

    def __init__(self, cls, *args, **kwargs) -> None:
        """
        RayClassWithInitArgsを初期化する
        実際のワーカーの生成は__call__メソッドで行う

        Args:
            cls: ワーカークラス
            *args: ワーカークラスの初期化引数
            **kwargs: ワーカークラスの初期化キーワード引数
        """

        logger.info(f"RayClassWithInitArgsを初期化 {cls=} {args=} {kwargs=}")

        super().__init__(cls, *args, **kwargs)

        # オプションの初期化
        self._options = {}

        # 追加リソースの初期化
        self._additional_resource = {}

    def set_additional_resource(self, additional_resource):
        self._additional_resource = additional_resource

    def update_options(self, options: Dict):
        self._options.update(options)

    def __call__(self, placement_group, placement_group_bundle_idx, use_gpu: bool = True, num_gpus=1, sharing_with=None) -> Any:
        """
        Rayのリモートワーカー（アクター）を生成する
        sharing_withが指定されている場合、共有先のワーカーと同じノードにアクターを配置する

        Args:
            placement_group: Rayのプレースメントグループ
            placement_group_bundle_idx: プレースメントグループのバンドルインデックス
            use_gpu (bool): GPUを使用するかどうかのフラグ
            num_gpus (int): 使用するGPUの数
            sharing_with: 共有先のワーカー
        Returns:
            Any: 生成されたRayのリモートワーカー
        """
        logger.info(f"RayClassWithInitArgsを呼び出し {placement_group=} {placement_group_bundle_idx=} {use_gpu=} {num_gpus=} {sharing_with=}")

        # 共有先のワーカーが指定されている場合
        if sharing_with is not None:

            # 共有先のワーカーのノードIDを取得
            target_node_id = ray.get(sharing_with.get_node_id.remote())

            # 共有先のGPU設定を取得
            cuda_visible_devices = ray.get(
                sharing_with.get_cuda_visible_devices.remote()
            )

            # スケジューリング戦略を作成
            # 強制的に共有先のノードにアクターを配置
            options = {
                "scheduling_strategy": \
                    NodeAffinitySchedulingStrategy(
                        node_id=target_node_id,
                        soft=False
                    )
            }

            # アクターを起動
            return self.cls.options(**options).remote(
                *self.args,
                cuda_visible_devices=cuda_visible_devices,
                **self.kwargs
            )

        # スケジューリング戦略を作成
        options = {
            "scheduling_strategy":
                PlacementGroupSchedulingStrategy(
                    placement_group=placement_group,
                    placement_group_bundle_index=placement_group_bundle_idx
                )
        }

        # 追加のオプションを更新
        options.update(self._options)

        # GPUを使用する場合、GPU数を設定
        if use_gpu:
            options["num_gpus"] = num_gpus

        # 追加リソースが指定されている場合、オプションに追加
        if len(self._additional_resource) > 1:
            for k, v in self._additional_resource.items():
                options[k] = v

        # アクターを起動
        return self.cls.options(**options).remote(*self.args, **self.kwargs)

#### RayResourcePool

In [ ]:
class RayResourcePool(ResourcePool):
    """
    Rayのプレースメントグループを管理するリソースプールクラス
    分散学習で、必要な計算リソースを事前に確保し、管理するためのクラス
    RayWorkerGroupで使用される
    """

    def __init__(self, process_on_nodes: List[int] = None, use_gpu: bool = True, name_prefix: str = "", max_colocate_count: int = 5, detached=False) -> None:
        """
        RayResourcePoolを初期化する

        Args:
            process_on_nodes (List[int], optional): 各ノードでのプロセス数のリスト
            use_gpu (bool, optional): GPUを使用するかどうかのフラグ
            name_prefix (str, optional): プレースメントグループ名の接頭辞
            max_colocate_count (int, optional): 各プロセスでコロケートできる最大数
            detached (bool, optional):
                デタッチドモードでプレースメントグループを作成するかどうかのフラグ
        Returns:
            None
        """

        logger.info(f"RayResourcePoolを初期化 {process_on_nodes=} {name_prefix=} {max_colocate_count=}")

        super().__init__(process_on_nodes, max_colocate_count)

        # GPU使用フラグを属性に保存
        self.use_gpu = use_gpu

        # プレースメントグループ名の接頭辞を属性に保存
        self.name_prefix = name_prefix

        # プレースメントグループの初期化
        self.pgs = None

        # デタッチドモードフラグを属性に保存
        self.detached = detached

    def get_placement_groups(self, strategy="STRICT_PACK", name=None):
        """
        リソースプールからRayのプレースメントグループを取得する

        Args:
            strategy (str, optional): プレースメントグループの配置戦略
            name (str, optional): プレースメントグループ名の接頭辞
        Returns:
            List[placement_group]: Rayのプレースメントグループのリスト
        """

        # 既にプレースメントグループが作成されている場合、それを返す
        if self.pgs is not None:
            return self.pgs

        # プレースメントグループ名の接頭辞を決定
        pg_name_prefix = name if name else \
            f"{self.name_prefix}verl_group_{'_'.join([
                str(count) for count in self._store
            ])}:"

        print(f"プレースメントグループ名の接頭辞 {pg_name_prefix=}")

        # プレースメントグループのバンドル（リソースの最小単位）を作成
        pg_scheme = [[{
            "CPU": self.max_collocate_count,
            "GPU": 1, # ワーカーに付き1つのGPU
        } if self.use_gpu else {
            "CPU": self.max_collocate_count, 
        } for _ in range(process_count)] for process_count in self._store
        ]

        # プレースメントグループの寿命を設定
        lifetime = 'detached' if self.detached else None

        # プレースメントグループを作成
        pgs = [
            placement_group(
                bundles=bundles,
                strategy=strategy,
                name=pg_name_prefix + str(idx), lifetime=lifetime
            ) for idx, bundles in enumerate(pg_scheme)
        ]

        # 全てのプレースメントグループが準備完了するまで待機
        ray.get([pg.ready() for pg in pgs])

        # 作成したプレースメントグループを属性に保存
        self.pgs = pgs

        return pgs

#### RayWorkerGroup

In [ ]:
class RayWorkerGroup(WorkerGroup):
    """
    複数の分散ワーカーを一括管理するための中心的なクラス

    - RayResourcePoolで確保したリソース上にワーカー（Rayアクター）を起動し管理
    - 分散実行を抽象化し、複数のワーカーに一斉にメソッドを実行させる
    """

    def __init__(
        self,
        resource_pool: RayResourcePool = None,
        ray_cls_with_init: RayClassWithInitArgs = None,
        bin_pack: bool = True,
        name_prefix: str = None,
        detached=False,
        worker_names=None,
        **kwargs
    ) -> None:
        """
        Args:
            resource_pool (RayResourcePool, optional):
                ワーカーを起動するためのリソースプール
            ray_cls_with_init (RayClassWithInitArgs, optional):
                ワーカーのクラスと初期化引数を保持するファクトリー
            bin_pack (bool, optional):
                プレースメントグループのバンドルを詰め込むかどうかのフラグ
            name_prefix (str, optional):
                ワーカー名の接頭辞
            detached (bool, optional):
                デタッチドモードでワーカーを起動するかどうかのフラグ
            worker_names (List[str], optional):
                デタッチドモードで起動したワーカーの名前のリスト
        """

        logger.info(f"RayWorkerGroupを初期化 {name_prefix=} {detached=} {worker_names=}")

        super().__init__(resource_pool=resource_pool, **kwargs)

        # RayClassWithInitArgsを属性に保存
        self.ray_cls_with_init = ray_cls_with_init

        # ワーカー名のリストを初期化
        self.name_prefix = get_random_string(length=6) if name_prefix is None else name_prefix

        # 
        if worker_names is not None:
            assert self._is_init_with_detached_workers
            self._worker_names = worker_names

        # 
        if self._is_init_with_detached_workers:
            self._init_with_detached_workers(worker_names=worker_names)

        else:
            # 新規にワーカーを起動
            self._init_with_resource_pool(
                resource_pool=resource_pool,
                ray_cls_with_init=ray_cls_with_init,
                bin_pack=bin_pack,
                detached=detached
            )

        if ray_cls_with_init is not None:
            # ワーカークラスが持っているメソッドをグループクラスのメソッドとして動的に登録する
            self._bind_worker_method(self.ray_cls_with_init.cls, func_generator)

    def _is_worker_alive(self, worker: ray.actor.ActorHandle):
        worker_state_dict = get_actor(worker._actor_id.hex())
        return worker_state_dict.get("state", "undefined") == "ALIVE" \
            if worker_state_dict is not None else False

    def _init_with_detached_workers(self, worker_names):
        workers = [ray.get_actor(name=name) for name in worker_names]
        self._workers = workers
        self._world_size = len(worker_names)

    def _init_with_resource_pool(self, resource_pool, ray_cls_with_init, bin_pack, detached):
        use_gpu = resource_pool.use_gpu

        strategy = "PACK"
        if bin_pack:
            strategy = "STRICT_PACK"
        pgs = resource_pool.get_placement_groups(strategy=strategy)
        world_size = resource_pool.world_size
        self._world_size = world_size
        # cia.add_kwarg("_world_size", world_size)
        num_gpus = 1 / resource_pool.max_collocate_count

        rank = -1
        for pg_idx, local_world_size in enumerate(resource_pool.store):
            pg = pgs[pg_idx]
            assert local_world_size <= pg.bundle_count, \
                f"when generating for {self.name_prefix}, for the "
            for local_rank in range(local_world_size):
                rank += 1

                # we pass in environment variable at option so that Worker can use environment variable to set
                env_vars = {
                    'WORLD_SIZE': str(world_size),
                    'RANK': str(rank),
                    'WG_PREFIX': self.name_prefix,
                    'WG_BACKEND': 'ray',
                    'RAY_LOCAL_WORLD_SIZE': str(local_world_size),
                    'RAY_LOCAL_RANK': str(local_rank),
                }
                if rank != 0:
                    env_vars['MASTER_ADDR'] = self._master_addr
                    env_vars['MASTER_PORT'] = self._master_port

                cia_name = type(ray_cls_with_init.cls).__name__
                match = re.search(r"ActorClass\(([^)]+)\)", cia_name)  # ray.remote(Obj) -> "ActorClass(Obj)"
                cia_name = match.group(1) if match else cia_name  # "ActorClass(Obj)" -> "Obj"
                name = f"{self.name_prefix}{cia_name}_{pg_idx}:{local_rank}"  # e.g. Worker_2:5

                ray_cls_with_init.update_options({'runtime_env': {'env_vars': env_vars}, 'name': name})

                if detached:
                    ray_cls_with_init.update_options({'lifetime': 'detached'})

                # create a worker
                worker = ray_cls_with_init(placement_group=pg,
                                           placement_group_bundle_idx=local_rank,
                                           use_gpu=use_gpu,
                                           num_gpus=num_gpus)
                self._workers.append(worker)
                self._worker_names.append(name)

                if rank == 0:
                    register_center_actor = None
                    for _ in range(120):
                        if f"{self.name_prefix}_register_center" not in list_named_actors():
                            time.sleep(1)
                        else:
                            register_center_actor = ray.get_actor(f"{self.name_prefix}_register_center")
                            break
                    assert register_center_actor is not None, f"failed to get register_center_actor: {self.name_prefix}_register_center in {list_named_actors(all_namespaces=True)}"
                    rank_zero_info = ray.get(register_center_actor.get_rank_zero_info.remote())
                    self._master_addr, self._master_port = rank_zero_info['MASTER_ADDR'], rank_zero_info['MASTER_PORT']
                    # print(f"rank_zero_info: {rank_zero_info}")
                    # print(f"master_addr: {self._master_addr}, master_port: {self._master_port}")

    @property
    def worker_names(self):
        return self._worker_names

    @classmethod
    def from_detached(cls, worker_names=None, ray_cls_with_init=None):
        worker_group = cls(resource_pool=None,
                           ray_cls_with_init=ray_cls_with_init,
                           name_prefix=None,
                           worker_names=worker_names)
        return worker_group

    def spawn(self, prefix_set):
        """
        物理的には同じワーカー群から、論理的に異なるワーカー群を生成する

        Args:
            prefix_set (Set[str]): ワーカー名の接頭辞のセット
        Returns:
            Dict[str, RayWorkerGroup]: 接頭辞ごとに分けられたRayWorkerGroupの辞書
        """

        def _rebind_actor_methods(worker_group, actor_name):
            """
            メソッド名の接頭辞を取り除き、元の名前にメソッドを再バインドする
            たとえば、"actor_update_policy"を"update_policy"に再バインドするなど

            Args:
                worker_group (RayWorkerGroup): ワーカーグループ
                actor_name (str): ワーカー名の接頭辞
            """

            # 接頭辞を定義
            prefix: str = actor_name + '_'

            # ワーカーグループの全てのメソッドを調べる
            for method_name in dir(worker_group):

                # メソッド名が接頭辞で始まる場合
                if method_name.startswith(prefix):

                    # 接頭辞を取り除いた元のメソッド名を取得（Python 3.9以降で有効）
                    # 例: actor_update_policy -> update_policy
                    original_method_name = method_name.removeprefix(prefix)

                    # メソッドを取得 
                    method = getattr(worker_group, method_name)

                    # 元のメソッド名で再バインド
                    setattr(worker_group, original_method_name, method)

        new_worker_group_dict = {}

        # 指定された役割ごとにループ（actor, refなど）
        for prefix in prefix_set:

            # 既に存在するワーカーに接続するだけの軽量なワーカーグループを作成
            new_worker_group = self.from_detached(
                worker_names=self._worker_names,
                ray_cls_with_init=self.ray_cls_with_init
            )

            # メソッドの再バインド
            _rebind_actor_methods(new_worker_group, prefix)

            # 新しいワーカーグループを辞書に追加
            new_worker_group_dict[prefix] = new_worker_group

        return new_worker_group_dict

    def execute_rank_zero_sync(self, method_name: str, *args, **kwargs):
        return ray.get(self.execute_all_async(method_name, **args, **kwargs))

    def execute_rank_zero_async(self, method_name: str, *args, **kwargs):
        remote_call = getattr(self._workers[0], method_name)
        return remote_call.remote(*args, **kwargs)

    def execute_rank_zero(self, method_name: str, *args, **kwargs):
        """
        ランク0のワーカーに対してメソッドを実行する

        Args:
            method_name (str): 呼び出すメソッドの名前
            *args: メソッドの引数
            **kwargs: メソッドのキーワード引数
        Returns:
            Any: ランク0のワーカーからのリモート呼び出しの結果
        """
        return self.execute_rank_zero_async(method_name, *args, **kwargs)

    def execute_all(self, method_name: str, *args, **kwargs):
        return self.execute_all_async(method_name, *args, **kwargs)

    def execute_all_sync(self, method_name: str, *args, **kwargs):
        return ray.get(self.execute_all_async(method_name, *args, **kwargs))

    def execute_all_async(self, method_name: str, *args, **kwargs):
        """
        全てのワーカーに対して非同期にメソッドを実行する

        Args:
            method_name (str): 呼び出すメソッドの名前
            *args: メソッドの引数
            **kwargs: メソッドのキーワード引数
        Returns:
            List: 各ワーカーからのリモート呼び出しの結果のリスト
        """
        # 这里我们假设，如果 args 和 kwargs 里面所有的参数都是 list，且所有的 list 长度都与 len(self._workers) 一致的话，我们会把
        # list 中的每一个分别发到对应的 worker 上去
        print(f"execute_all_async: method {method_name}({args}, {kwargs})")
        length = len(self._workers)

        if all(isinstance(arg, list) for arg in args) and \
            all(isinstance(kwarg, list) for kwarg in kwargs.values()):

            if all(len(arg) == length for arg in args) and \
                all(len(kwarg) == length for kwarg in kwargs.values()):
                # print(f"splitting args and kwargs into {length} shards")
                result = []

                for i in range(length):
                    sliced_args = tuple(arg[i] for arg in args)
                    sliced_kwargs = {k: v[i] for k, v in kwargs.items()}
                    remote_call = getattr(self._workers[i], method_name)
                    result.append(remote_call.remote(*sliced_args, **sliced_kwargs))
                return result

        # 全てのワーカーに対してメソッドを非同期に実行
        return [getattr(worker, method_name).remote(*args, **kwargs) for worker in self._workers]

    @property
    def master_address(self):
        return self._master_addr

    @property
    def master_port(self):
        return self._master_port

    @property
    def workers(self):
        return self._workers

    @property
    def world_size(self):
        return self._world_size

### トレーナー

#### カスタム型

In [ ]:
WorkerType = Type[Worker]

In [ ]:
class Role(Enum):
    """
    ワーカーの役割を定義する列挙型
    """
    Actor = 0
    Rollout = 1
    ActorRollout = 2
    Critic = 3
    RefPolicy = 4
    RewardModel = 5
    ActorRolloutRef = 6

In [ ]:
def masked_mean(values, mask, axis=None):
    """
    マスク付きのテンソルの平均を計算する

    Args:
        values (torch.Tensor): 値のテンソル
        mask (torch.Tensor): マスクのテンソル（0または1の値を持つ）
        axis (int or tuple of int, optional): 平均を計算する軸
    Returns:
        torch.Tensor: マスク付きの平均値のテンソル
    """
    return (values * mask).sum(axis=axis) / mask.sum(axis=axis)

In [ ]:
@contextmanager
def _timer(name: str, timing_raw: Dict[str, float]):
    """
    処理時間を計測するコンテキストマネージャ

    Args:
        name (str): 計測する処理の名前
        timing_raw (Dict[str, float]): 計測結果を保存する辞書
    """
    with Timer(name=name, logger=None) as timer:
        yield # 呼び出し元のwithブロック内の処理が実行される

    timing_raw[name] = timer.last

In [ ]:
def apply_kl_penalty(data: DataProto, kl_ctrl: core_algos.AdaptiveKLController, kl_penalty='kl'):
    """
    KLペナルティを適用してトークンレベルの報酬を計算する
    PPOで元のポリシーからの乖離を制御するために使用される

    Args:
        data (DataProto): 入力データ
        kl_ctrl (core_algos.AdaptiveKLController): KLコントローラー
        kl_penalty (str): KLペナルティのタイプ
    Returns:
        data (DataProto): KLペナルティが適用されたデータ
        metrics (Dict[str, float]): KL関連のメトリクス
    """

    # 生成された応答を取得
    responses = data.batch['responses']

    # 応答の長さを取得
    response_length = responses.size(1)

    # トークンレベルのスコアを取得
    token_level_scores = data.batch['token_level_scores']

    # バッチサイズを取得
    batch_size = data.batch.batch_size[0]

    # アテンションマスクを取得
    attention_mask = data.batch['attention_mask']

    # 応答部分のアテンションマスクを取得
    response_mask = attention_mask[:, -response_length:]

    # compute kl between ref_policy and current policy

    # 参照モデルの対数確率が存在する場合
    if 'ref_log_prob' in data.batch.keys():

        # KLダイバージェンスを計算
        kld = core_algos.kl_penalty(
            data.batch['old_log_probs'],
            data.batch['ref_log_prob'],
            kl_penalty=kl_penalty
        )  # (batch_size, response_length)

        # マスクを適用（パディングは0にする）
        kld = kld * response_mask

        # KL係数を取得
        beta = kl_ctrl.value

    # 参照モデルの対数確率が存在しない場合
    else:
        # KLペナルティを適用しない
        beta = 0

        # KLダイバージェンスをゼロに設定
        kld = torch.zeros_like(response_mask, dtype=torch.float32)

    # トークンレベルの報酬を計算
    # 最終報酬 = 報酬モデルのスコア - 係数 * KLダイバージェンス
    token_level_rewards = token_level_scores - beta * kld

    # バッチ全体の平均KLダイバージェンスを計算
    current_kl = masked_mean(kld, mask=response_mask, axis=-1)
    current_kl = torch.mean(current_kl, dim=0).item()

    # KLコントローラーを更新
    # KLダイバージェンスに基づいてKL係数betaを調整
    kl_ctrl.update(current_kl=current_kl, n_steps=batch_size)

    # トークンレベルの報酬をデータに保存
    data.batch['token_level_rewards'] = token_level_rewards

    # KL関連のメトリクスを作成
    metrics = {'critic/kl': current_kl, 'critic/kl_coeff': beta}

    return data, metrics

In [ ]:
def compute_advantage(data: DataProto, adv_estimator, gamma=1.0, lam=1.0, num_repeat=1):
    """
    強化学習におけるアドバンテージとリターン（割引累積報酬）を計算する

    Args:
        data (DataProto): 入力データ
        adv_estimator (str): アドバンテージ推定器のタイプ（'gae'または'grpo'）
        gamma (float): 割引率
        lam (float): GAEのλパラメータ
        num_repeat (int): データの繰り返し数（未使用）
    Returns:
        data (DataProto): アドバンテージとリターンが計算されたデータ
    """

    # GAE（Generalized Advantage Estimation）を使用する場合
    if adv_estimator == 'gae':

        # クリティックが予測した状態価値
        values = data.batch['values']

        # 生成された応答を取得
        responses = data.batch['responses']

        # 応答の長さを取得
        response_length = responses.size(-1)

        # アテンションマスクを取得
        attention_mask = data.batch['attention_mask']

        # 応答部分のアテンションマスクを取得
        response_mask = attention_mask[:, -response_length:]

        # トークンレベルの報酬を取得
        token_level_rewards = data.batch['token_level_rewards']

        # アドバンテージとリターンを計算
        advantages, returns = core_algos.compute_gae_advantage_return(
            token_level_rewards=token_level_rewards, # トークンレベルの報酬
            values=values, # クリティックの状態価値予測
            eos_mask=response_mask, # 応答部分のマスク
            gamma=gamma, # 割引率
            lam=lam # GAEのλパラメータ（未来の報酬の影響度を調整）
        )

        # アドバンテージをデータに保存
        data.batch['advantages'] = advantages

        # リターンをデータに保存
        data.batch['returns'] = returns

    # GRPO（Generalized Reward Policy Optimization）を使用する場合
    elif adv_estimator == 'grpo':

        # トークンレベルの報酬を取得
        token_level_rewards = data.batch['token_level_rewards']

        # データがどのサンプルに対応するかのインデックスを取得
        index = data.non_tensor_batch['uid']

        # 生成された応答を取得
        responses = data.batch['responses']

        # 応答の長さを取得
        response_length = responses.size(-1)

        # アテンションマスクを取得
        attention_mask = data.batch['attention_mask']

        # 応答部分のアテンションマスクを取得
        response_mask = attention_mask[:, -response_length:]

        # アドバンテージとリターンを計算
        advantages, returns = core_algos.compute_grpo_outcome_advantage(
            token_level_rewards=token_level_rewards,
            eos_mask=response_mask,
            index=index
        )

        # アドバンテージをデータに保存
        data.batch['advantages'] = advantages

        # リターンをデータに保存
        data.batch['returns'] = returns
    else:
        raise NotImplementedError
    return data

In [ ]:
def reduce_metrics(metrics: dict):
    """
    複数のメトリクスを平均化する
    分散学習では、複数のGPUから学習結果がリスト形式で返されるため

    Args:
        metrics (dict): メトリクスの辞書
    Returns:
        dict: 平均化されたメトリクスの辞書
    """

    # 各メトリクスの辞書のキーとバリューでループ
    for key, val in metrics.items():

        # 平均を計算して更新
        metrics[key] = np.mean(val)

    return metrics

In [ ]:
def _compute_response_info(batch):
    """
    バッチから、プロンプトと回答のトークン数と、回答部分のマスクを計算する

    Args:
        batch (DataProto): 入力バッチ
    Returns:
        dict: 回答のマスク、プロンプト長、回答のトークン数を含む辞書
    """

    # バッチから回答の長さを取得
    response_length = batch.batch['responses'].shape[-1]

    # バッチからプロンプト部分のマスクを取得
    prompt_mask = batch.batch['attention_mask'][:, :-response_length]

    # バッチから回答部分のマスクを取得
    response_mask = batch.batch['attention_mask'][:, -response_length:]

    # プロンプトのトークン数を計算
    prompt_length = prompt_mask.sum(-1).float()

    # 回答のトークン数を計算
    response_length = response_mask.sum(-1).float()  # (batch_size,)

    # 結果を辞書で返す
    return dict(
        response_mask=response_mask, # 回答部分のマスク
        prompt_length=prompt_length, # プロンプトのトークン数
        response_length=response_length, # 回答のトークン数
    )

In [ ]:
def compute_data_metrics(batch, use_critic=True):
    """
    バッチから様々なメトリクスを計算する
    学習進捗のモニタリングに使用される

    Args:
        batch (DataProto): 入力バッチ
        use_critic (bool): クリティックを使用するかどうかのフラグ
    Returns:
        dict: 計算されたメトリクスの辞書
    """

    # シーケンス全体のスコアを計算（報酬モデルのスコア）
    sequence_score = batch.batch['token_level_scores'].sum(-1)

    # シーケンス全体の報酬を計算（報酬モデルのスコアからKLペナルティを引いたもの）
    sequence_reward = batch.batch['token_level_rewards'].sum(-1)

    # アドバンテージを取得
    advantages = batch.batch['advantages']

    # リターンを取得
    returns = batch.batch['returns']

    # バッチから回答の長さを取得
    max_response_length = batch.batch['responses'].shape[-1]

    # プロンプト部分のマスクを取得
    prompt_mask = batch.batch['attention_mask'][:, :-max_response_length].bool()

    # 回答部分のマスクを取得
    response_mask = batch.batch['attention_mask'][:, -max_response_length:].bool()

    # プロンプト部分の最大長を取得
    max_prompt_length = prompt_mask.size(-1)

    # プロンプトと回答の情報を計算
    response_info = _compute_response_info(batch)

    # プロンプトのトークン数を取得 
    prompt_length = response_info['prompt_length']

    # 回答のトークン数を取得
    response_length = response_info['response_length']

    # マスクを適用して有効なアドバンテージを取得
    valid_adv = torch.masked_select(advantages, response_mask)

    # マスクを適用して有効なリターンを取得
    valid_returns = torch.masked_select(returns, response_mask)

    # クリティックを使用する場合、価値関数の分散を計算
    if use_critic:
        values = batch.batch['values']
        valid_values = torch.masked_select(values, response_mask)
        return_diff_var = torch.var(valid_returns - valid_values)
        return_var = torch.var(valid_returns)

    # 各種メトリクスを計算して辞書にまとめる
    metrics = {
        # score
        'critic/score/mean':
            torch.mean(sequence_score).detach().item(),
        'critic/score/max':
            torch.max(sequence_score).detach().item(),
        'critic/score/min':
            torch.min(sequence_score).detach().item(),
        # reward
        'critic/rewards/mean':
            torch.mean(sequence_reward).detach().item(),
        'critic/rewards/max':
            torch.max(sequence_reward).detach().item(),
        'critic/rewards/min':
            torch.min(sequence_reward).detach().item(),
        # adv
        'critic/advantages/mean':
            torch.mean(valid_adv).detach().item(),
        'critic/advantages/max':
            torch.max(valid_adv).detach().item(),
        'critic/advantages/min':
            torch.min(valid_adv).detach().item(),
        # returns
        'critic/returns/mean':
            torch.mean(valid_returns).detach().item(),
        'critic/returns/max':
            torch.max(valid_returns).detach().item(),
        'critic/returns/min':
            torch.min(valid_returns).detach().item(),
        **({
            # values
            'critic/values/mean': torch.mean(valid_values).detach().item(),
            'critic/values/max': torch.max(valid_values).detach().item(),
            'critic/values/min': torch.min(valid_values).detach().item(),
            # vf explained var
            'critic/vf_explained_var': (1.0 - return_diff_var / (return_var + 1e-5)).detach().item(),
        } if use_critic else {}),

        # response length
        'response_length/mean':
            torch.mean(response_length).detach().item(),
        'response_length/max':
            torch.max(response_length).detach().item(),
        'response_length/min':
            torch.min(response_length).detach().item(),
        'response_length/clip_ratio':
            torch.mean(torch.eq(response_length, max_response_length).float()).detach().item(),
        # prompt length
        'prompt_length/mean':
            torch.mean(prompt_length).detach().item(),
        'prompt_length/max':
            torch.max(prompt_length).detach().item(),
        'prompt_length/min':
            torch.min(prompt_length).detach().item(),
        'prompt_length/clip_ratio':
            torch.mean(torch.eq(prompt_length, max_prompt_length).float()).detach().item(),
    }
    return metrics

In [ ]:
def compute_timing_metrics(batch, timing_raw):
    """
    バッチから処理にかかった時間のメトリクスを計算する

    Args:
        batch (DataProto): 入力バッチ
        timing_raw (Dict[str, float]): 生の処理時間データ
    Returns:
        dict: 計算された時間メトリクスの辞書
    """

    # プロンプトと回答の情報を計算
    response_info = _compute_response_info(batch)

    # プロンプトのトークン数を計算
    num_prompt_tokens = torch.sum(response_info['prompt_length']).item()

    # 回答のトークン数を計算
    num_response_tokens = torch.sum(response_info['response_length']).item()

    # 全体のトークン数を計算
    num_overall_tokens = num_prompt_tokens + num_response_tokens

    # 各セクションごとのトークン数を辞書にまとめる
    num_tokens_of_section = {
        'gen': num_response_tokens, # 生成時は回答部分のトークン数
        **{
            name: num_overall_tokens for name in ['ref', 'values', 'adv', 'update_critic', 'update_actor']
        }, # その他は全体のトークン数
    }

    return {
        **{
            f'timing_s/{name}': value for name, value in timing_raw.items()
        }, # 各セクションごとの生の時間（秒）
        **{
            f'timing_per_token_ms/{name}': timing_raw[name] * 1000 / num_tokens_of_section[name] for name in set(num_tokens_of_section.keys(
            )) & set(timing_raw.keys())
        }, # 各セクションごとのトークンあたりの時間（ミリ秒）
    }

#### ResourcePoolManager

In [ ]:
@dataclass
class ResourcePoolManager:
    """
    物理的なGPUリソースと論理的な役割の対応関係を管理するクラス
    """

    # 物理的なリソースの定義
    # 例: {'actor_pool': [4, 4], 'critic_pool': [2, 2]}
    # この場合、actor_poolは2ノードで書くノードに4つのGPUを持つ
    resource_pool_spec: dict[str, list[int]]

    # 役割とリソースプール名の対応関係
    # 例: {Role.Actor: 'actor_pool', Role.Critic: 'critic_pool'}
    mapping: dict[Role, str]

    # 実際に作成されたリソースプールの辞書
    resource_pool_dict: dict[str, RayResourcePool] = field(default_factory=dict)

    def create_resource_pool(self):
        """
        リソースプールを作成する
        """

        # resource_pool_specに基づいてリソースプールを初期化
        for resource_pool_name, process_on_nodes in self.resource_pool_spec.items():

            # RayResourcePoolを作成
            resource_pool = RayResourcePool(
                process_on_nodes=process_on_nodes, # ノードごとのプロセス数
                use_gpu=True, # GPUを使用
                max_colocate_count=1, # 1つのリソースプールに付き1つの巨大な分散モデルグループを配置
                name_prefix=resource_pool_name # プレースメントグループ名の接頭辞
            )

            # リソースプールを辞書に保存
            self.resource_pool_dict[resource_pool_name] = resource_pool

    def get_resource_pool(self, role: Role) -> RayResourcePool:
        """
        指定された役割に対応するリソースプールを取得する

        Args:
            role (Role): ワーカーの役割
        Returns:
            RayResourcePool: 指定された役割に対応するリソースプール
        """
        return self.resource_pool_dict[self.mapping[role]]

#### RayPPOTrainer

In [ ]:
class RayPPOTrainer(object):
    """
    PPOトレーナークラス
    """

    def __init__(self, config, tokenizer, role_worker_mapping: dict[Role, WorkerType], resource_pool_manager: ResourcePoolManager, ray_worker_group_cls: RayWorkerGroup = RayWorkerGroup, reward_fn=None, val_reward_fn=None):
        """
        Args:
            config: PPOトレーニングの設定
            tokenizer: トークナイザー
            role_worker_mapping (dict[Role, WorkerType]): 役割とワーカータイプ
            resource_pool_manager (ResourcePoolManager): リソースプールマネージャー
            ray_worker_group_cls (RayWorkerGroup): Rayワーカーグループクラス
            reward_fn: 報酬関数
            val_reward_fn: 検証用報酬関数
        """
        print(f"PPOトレーナーを初期化")

        # トークナイザーを属性に保存
        self.tokenizer = tokenizer

        # 設定を属性に保存
        self.config = config

        # 報酬関数を属性に保存
        self.reward_fn = reward_fn

        # 検証用報酬関数を属性に保存
        self.val_reward_fn = val_reward_fn

        # ハイブリッドエンジンの使用有無を設定から取得
        # Trueの場合、ActorRolloutワーカーが存在することを意味する
        self.hybrid_engine = config.actor_rollout_ref.hybrid_engine

        assert self.hybrid_engine, 'Currently, only support hybrid engine'

        if self.hybrid_engine:
            assert Role.ActorRollout in role_worker_mapping, f'{role_worker_mapping.keys()=}'

        # 役割とワーカータイプのマッピングを属性に保存
        self.role_worker_mapping = role_worker_mapping

        # リソースプールマネージャーを属性に保存
        self.resource_pool_manager = resource_pool_manager

        # 参照ポリシーの使用有無を設定
        self.use_reference_policy = Role.RefPolicy in role_worker_mapping

        # 報酬モデルの使用有無を設定
        self.use_rm = Role.RewardModel in role_worker_mapping

        # Rayワーカーグループクラスを属性に保存
        self.ray_worker_group_cls = ray_worker_group_cls

        # 参照ポリシーを使用する場合
        if self.use_reference_policy:

            # 固定のKLコントローラーを使用する場合
            if config.algorithm.kl_ctrl.type == 'fixed':

                # 固定のKLコントローラーを初期化
                self.kl_ctrl = core_algos.FixedKLController(
                    kl_coef=config.algorithm.kl_ctrl.kl_coef
                )

            # 適応的なKLコントローラーを使用する場合
            elif config.algorithm.kl_ctrl.type == 'adaptive':

                assert config.algorithm.kl_ctrl.horizon > 0, \
                    f'horizon must be larger than 0. Got {config.critic.kl_ctrl.horizon}'

                # 適応的なKLコントローラーを初期化
                self.kl_ctrl = core_algos.AdaptiveKLController(
                    init_kl_coef=config.algorithm.kl_ctrl.kl_coef,
                    target_kl=config.algorithm.kl_ctrl.target_kl,
                    horizon=config.algorithm.kl_ctrl.horizon
                )
            else:
                raise NotImplementedError

        # 参照ポリシーを使用しない場合、KLコントローラーのKL係数を0に設定
        else:
            self.kl_ctrl = core_algos.FixedKLController(kl_coef=0.)

        # データローダーを作成
        self._create_dataloader()

    def _create_dataloader(self):
        """
        訓練と検証のデータローダーを作成する
        """

        # 訓練データセットを初期化
        self.train_dataset = RLHFDataset(
            parquet_files=self.config.data.train_files,
            tokenizer=self.tokenizer,
            prompt_key=self.config.data.prompt_key,
            max_prompt_length=self.config.data.max_prompt_length,
            filter_prompts=True,
            return_raw_chat=self.config.data.get('return_raw_chat', False),
            truncation='error'
        )

        # 訓練データローダーを初期化
        self.train_dataloader = DataLoader(
            dataset=self.train_dataset,
            batch_size=self.config.data.train_batch_size,
            shuffle=True,
            drop_last=True,
            collate_fn=collate_fn
        )

        # 検証データセットを初期化 
        self.val_dataset = RLHFDataset(
            parquet_files=self.config.data.val_files,
            tokenizer=self.tokenizer,
            prompt_key=self.config.data.prompt_key,
            max_prompt_length=self.config.data.max_prompt_length,
            filter_prompts=True,
            return_raw_chat=self.config.data.get('return_raw_chat', False),
            truncation='error'
        )

        # 検証データローダーを初期化
        self.val_dataloader = DataLoader(
            dataset=self.val_dataset,
            batch_size=len(self.val_dataset),
            shuffle=True,
            drop_last=True,
            collate_fn=collate_fn
        )

        assert len(self.train_dataloader) >= 1
        assert len(self.val_dataloader) >= 1

        print(f'訓練データローダーのバッチ数: {len(self.train_dataloader)}')
        print(f'検証データローダーのバッチ数: {len(self.val_dataloader)}')

        # デフォルトの総訓練ステップ数を計算
        # エポック数 * バッチ数
        total_training_steps = len(self.train_dataloader) * self.config.trainer.total_epochs

        # 設定で総訓練ステップ数が指定されている場合
        if self.config.trainer.total_training_steps is not None:
            # 設定の値で上書き
            total_training_steps = self.config.trainer.total_training_steps

        # 総訓練ステップ数を属性に保存
        self.total_training_steps = total_training_steps

        print(f'総訓練ステップ数: {self.total_training_steps}')

        # 設定を構造化モードに設定
        OmegaConf.set_struct(self.config, True)

        # 設定を更新
        with open_dict(self.config):

            # アクター、ロールアウト、参照ポリシーの最適化関数の総訓練ステップ数を設定
            self.config.actor_rollout_ref.actor.optim.total_training_steps = total_training_steps

            # クリティックの最適化関数の総訓練ステップ数を設定
            self.config.critic.optim.total_training_steps = total_training_steps

    def _validate(self):
        """
        検証データセットでモデルを評価する
        ルールベースの報酬モデルでのみ評価を行う
        """

        reward_tensor_lst = []

        data_source_lst = []

        # 検証データローダーでループ
        for test_data in self.val_dataloader:

            # 検証データをDataProtoに変換
            test_batch = DataProto.from_single_dict(test_data)

            # 報酬モデルが有効な場合はスキップ
            if self.config.reward_model.enable and \
                test_batch[0].non_tensor_batch['reward_model']['style'] == 'model':
                return {}

            # 生成のために必要な情報を分離
            test_gen_batch = test_batch.pop(['input_ids', 'attention_mask', 'position_ids'])

            # 生成設定をメタ情報に追加
            test_gen_batch.meta_info = {
                'eos_token_id': self.tokenizer.eos_token_id,
                'pad_token_id': self.tokenizer.pad_token_id,
                'recompute_log_prob': False,
                'do_sample': False,
                'validate': True,
            }

            # pad to be divisible by dp_size

            # 
            test_gen_batch_padded, pad_size = pad_dataproto_to_divisor(
                test_gen_batch,
                self.actor_rollout_wg.world_size
            )

            test_output_gen_batch_padded = self.actor_rollout_wg.generate_sequences(
                test_gen_batch_padded
            )

            # unpad
            test_output_gen_batch = unpad_dataproto(
                test_output_gen_batch_padded, pad_size=pad_size
            )

            print('validation generation end')

            test_batch = test_batch.union(test_output_gen_batch)

            # evaluate using reward_function
            # for certain reward function (e.g. sandbox), the generation can overlap with reward
            reward_tensor = self.val_reward_fn(test_batch)

            reward_tensor_lst.append(reward_tensor)
            data_source_lst.append(test_batch.non_tensor_batch.get('data_source', ['unknown'] * reward_tensor.shape[0]))

        reward_tensor = torch.cat(reward_tensor_lst, dim=0).sum(-1).cpu()  # (batch_size,)
        data_sources = np.concatenate(data_source_lst, axis=0)
        # evaluate test_score based on data source
        data_source_reward = {}
        for i in range(reward_tensor.shape[0]):
            data_source = data_sources[i]
            if data_source not in data_source_reward:
                data_source_reward[data_source] = []
            data_source_reward[data_source].append(reward_tensor[i].item())

        metric_dict = {}
        for data_source, rewards in data_source_reward.items():
            metric_dict[f'val/test_score/{data_source}'] = np.mean(rewards)

        return metric_dict

    def init_workers(self):
        """
        Rayを使ってリソースプールを作成し、各役割のワーカーを初期化する
        """

        # リソースプール（GPUのグループ）の作成
        self.resource_pool_manager.create_resource_pool()

        # リソースプールごとにワーカークラスを格納する辞書を初期化
        self.resource_pool_to_cls = {
            pool: {} for pool in self.resource_pool_manager.resource_pool_dict.values()
        }

        # ハイブリッドエンジンを使用する場合
        # True
        if self.hybrid_engine:

            # アクターロールアウトのリソースプールを取得
            resource_pool = self.resource_pool_manager.get_resource_pool(Role.ActorRollout)

            # 
            actor_rollout_cls = RayClassWithInitArgs(
                cls=self.role_worker_mapping[Role.ActorRollout],
                config=self.config.actor_rollout_ref,
                role='actor_rollout'
            )

            self.resource_pool_to_cls[resource_pool]['actor_rollout'] = actor_rollout_cls
        else:
            raise NotImplementedError

        # GAE（Generalized Advantage Estimation）を使用する場合
        if self.config.algorithm.adv_estimator == 'gae':

            # クリティックのリソースプールを取得
            resource_pool = self.resource_pool_manager.get_resource_pool(Role.Critic)

            # クリティックのワーカークラスを作成
            critic_cls = RayClassWithInitArgs(
                cls=self.role_worker_mapping[Role.Critic],
                config=self.config.critic # クリティックの設定
            )

            # クリティックのワーカークラスをリソースプールに登録
            self.resource_pool_to_cls[resource_pool]['critic'] = critic_cls

            # クリティック使用フラグを有効化
            self.use_critic = True

        # GRPO（Generalized Reward Policy Optimization）を使用する場合
        elif self.config.algorithm.adv_estimator == 'grpo':

            # クリティック使用フラグを無効化
            self.use_critic = False
        else:
            raise NotImplementedError

        # 参照ポリシーを使用する場合
        if self.use_reference_policy:

            # 参照ポリシーのリソースプールを取得
            resource_pool = self.resource_pool_manager.get_resource_pool(Role.RefPolicy)

            # 参照ポリシーのワーカークラスを作成
            ref_policy_cls = RayClassWithInitArgs(
                self.role_worker_mapping[Role.RefPolicy],
                config=self.config.actor_rollout_ref, # 参照ポリシーはアクターロールアウトと同じ設定を使用
                role='ref'
            )

            # 参照ポリシーのワーカークラスをリソースプールに登録
            self.resource_pool_to_cls[resource_pool]['ref'] = ref_policy_cls

        # 報酬モデルを使用する場合
        if self.use_rm:

            # 報酬モデルのリソースプールを取得
            resource_pool = self.resource_pool_manager.get_resource_pool(Role.RewardModel)

            # 報酬モデルのワーカークラスを作成
            rm_cls = RayClassWithInitArgs(
                self.role_worker_mapping[Role.RewardModel],
                config=self.config.reward_model # 報酬モデルの設定
            )

            # 報酬モデルのワーカークラスをリソースプールに登録
            self.resource_pool_to_cls[resource_pool]['rm'] = rm_cls


        # ワーカーグループを初期化
        all_wg = {}

        self.wg_dicts = []

        # リソースプールとワーカークラスの辞書でループ
        for resource_pool, class_dict in self.resource_pool_to_cls.items():

            # 複数の役割をまとめた複合クラスを作成
            worker_dict_cls = create_colocated_worker_cls(class_dict=class_dict)

            # ワーカーグループを作成
            wg_dict = self.ray_worker_group_cls(
                resource_pool=resource_pool,
                ray_cls_with_init=worker_dict_cls
            )

            # Rayを使ってワーカーを起動
            spawn_wg = wg_dict.spawn(prefix_set=class_dict.keys())

            # すべてのワーカーグループを1つの辞書にまとめる
            all_wg.update(spawn_wg)

            self.wg_dicts.append(wg_dict)

        # クリティックを使用する場合
        if self.use_critic:

            # クリティックのワーカーグループを取得
            self.critic_wg = all_wg['critic']

            # クリティックを初期化
            self.critic_wg.init_model()

        # 参照ポリシーを使用する場合
        if self.use_reference_policy:

            # 参照ポリシーのワーカーグループを取得
            self.ref_policy_wg = all_wg['ref']

            # 参照ポリシーを初期化
            self.ref_policy_wg.init_model()


        # リワードモデルを使用する場合
        if self.use_rm:

            # リワードモデルのワーカーグループを取得
            self.rm_wg = all_wg['rm']

            # リワードモデルを初期化
            self.rm_wg.init_model()

        # アクターロールアウトのワーカーグループを取得
        self.actor_rollout_wg = all_wg['actor_rollout']

        # アクターロールアウトを初期化（vLLMは大量にメモリを使用するため最後に初期化する）
        self.actor_rollout_wg.init_model()

    def _save_checkpoint(self):
        """
        モデルのチェックポイントを保存する
        """

        # アクターのチェックポイントのパスを設定
        actor_local_path = os.path.join(
            self.config.trainer.default_local_dir,
            'actor',
            f'global_step_{self.global_steps}'
        )

        # HDFSの設定がある場合は、リモートパスを設定
        actor_remote_path = None if self.config.trainer.default_hdfs_dir is None \
            else os.path.join(self.config.trainer.default_hdfs_dir, 'actor')

        # アクターのチェックポイントを保存
        self.actor_rollout_wg.save_checkpoint(actor_local_path, actor_remote_path)

        # クリティックを使用する場合
        if self.use_critic:

            # クリティックのチェックポイントのパスを設定
            critic_local_path = os.path.join(
                self.config.trainer.default_local_dir,
                'critic',
                f'global_step_{self.global_steps}'
            )

            # HDFSの設定がある場合は、リモートパスを設定
            critic_remote_path = None if self.config.trainer.default_hdfs_dir is None \
                else os.path.join(self.config.trainer.default_hdfs_dir, 'critic')

            # クリティックのチェックポイントを保存
            self.critic_wg.save_checkpoint(critic_local_path, critic_remote_path)

    def _balance_batch(self, batch: DataProto, metrics, logging_prefix='global_seqlen'):
        """
        GPUごとのトークン数が均等になるように、データの並び順をシャッフルして負荷分散を行う

        Args:
            batch (DataProto): 入力バッチ
            metrics (dict): メトリクスを格納する辞書
            logging_prefix (str): ログに使用するプレフィックス
        Returns:
            None: バッチはインプレースで並び替えられる
        """

        # バッチからアテンションマスクを取得
        attention_mask = batch.batch['attention_mask']

        # バッチサイズを取得
        batch_size = attention_mask.shape[0]

        # 各シーケンスの有効なトークン数を計算
        # (train_batch_size,)
        global_seqlen_lst = batch.batch['attention_mask'].view(batch_size, -1).sum(-1).tolist()

        # ワールドサイズ（使用しているGPUの数）を取得
        world_size = self.actor_rollout_wg.world_size

        # シーケンス長に基づいてバランスの取れたパーティションを計算
        global_partition_lst = get_seqlen_balanced_partitions(
            global_seqlen_lst,
            k_partitions=world_size, # GPUの数
            equal_size=True # 各パーティションのサイズを均等にする
        )

        # グローバルなインデックスを計算
        global_idx = torch.tensor([j for partition in global_partition_lst for j in partition])

        # バッチをグローバルインデックスに基づいて並び替え
        batch.reorder(global_idx)

        # 並び替え後のシーケンス長リストを再計算
        global_balance_stats = log_seqlen_unbalance(
            seqlen_list=global_seqlen_lst,
            partitions=global_partition_lst,
            prefix=logging_prefix
        )

        # メトリクスに負荷分散の統計情報を追加
        metrics.update(global_balance_stats)

    def fit(self):
        """
        学習のメインループ
        """

        logger = Tracking(
            project_name=self.config.trainer.project_name,
            experiment_name=self.config.trainer.experiment_name,
            default_backend=self.config.trainer.logger,
            config=OmegaConf.to_container(self.config, resolve=True)
        )

        self.global_steps = 0

        # perform validation before training
        # currently, we only support validation using the reward_function.
        if self.val_reward_fn is not None and self.config.trainer.get('val_before_train', True):
            val_metrics = self._validate()
            pprint(f'Initial validation metrics: {val_metrics}')
            logger.log(data=val_metrics, step=self.global_steps)
            if self.config.trainer.get('val_only', False):
                return

        # we start from step 1
        self.global_steps += 1

        # エポックごとにループ
        for epoch in range(self.config.trainer.total_epochs):

            # バッチごとにループ
            for batch_dict in self.train_dataloader:

                print(f'エポック {epoch}, ステップ {self.global_steps}')

                metrics = {}

                timing_raw = {}

                # バッチデータをDataProtoに変換
                batch: DataProto = DataProto.from_single_dict(batch_dict)

                # 生成に必要な情報を分離
                gen_batch = batch.pop(batch_keys=['input_ids', 'attention_mask', 'position_ids'])

                # ステップの処理時間を記録
                with _timer('step', timing_raw):

                    # データ生成の処理時間を計測
                    with _timer('gen', timing_raw):

                        # データ生成（ロールアウト）
                        gen_batch_output = self.actor_rollout_wg.generate_sequences(gen_batch)

                    # 元のプロンプト1つ1つに対して一意なUIDを生成
                    batch.non_tensor_batch['uid'] = np.array(
                        [str(uuid.uuid4()) for _ in range(len(batch.batch))],
                        dtype=object
                    )

                    # ロールアウトの数だけプロンプトを複製
                    batch = batch.repeat(
                        repeat_times=self.config.actor_rollout_ref.rollout.n,
                        interleave=True
                    )

                    # 複製したプロンプトと生成結果を結合
                    batch = batch.union(gen_batch_output)

                    # 各GPUが処理するトークン数が均等になるようにバッチを並び替える（負荷分散）
                    self._balance_batch(batch, metrics=metrics)

                    # グローバルなトークン数を計算してメタ情報に保存
                    batch.meta_info['global_token_num'] = torch.sum(
                        batch.batch['attention_mask'], dim=-1
                    ).tolist()

                    # 参照ポリシーを使用する場合
                    if self.use_reference_policy:

                        # 参照ポリシーの処理時間を計測
                        with _timer('ref', timing_raw):

                            # 参照ポリシーの対数確率を計算
                            ref_log_prob = self.ref_policy_wg.compute_ref_log_prob(batch)

                            batch = batch.union(ref_log_prob)

                    # クリティックを使用する場合
                    if self.use_critic:

                        # クリティックの処理時間を計測
                        with _timer('values', timing_raw):

                            # クリティックに現在の状態の価値を予測させる（GAE用）
                            values = self.critic_wg.compute_values(batch)

                            # 
                            batch = batch.union(values)

                    with _timer('adv', timing_raw):

                        # 報酬モデルを使用する場合
                        if self.use_rm:
                            # 報酬モデルのスコアを計算
                            reward_tensor = self.rm_wg.compute_rm_score(batch)
                            batch = batch.union(reward_tensor)

                        # ルールベースの報酬関数と組み合わせる
                        reward_tensor = self.reward_fn(batch)

                        # トークンレベルのスコアをバッチに保存
                        batch.batch['token_level_scores'] = reward_tensor

                        # KLペナルティを適用する場合
                        if not self.config.actor_rollout_ref.actor.use_kl_loss:

                            # KLペナルティを計算して適用
                            batch, kl_metrics = apply_kl_penalty(
                                batch,
                                kl_ctrl=self.kl_ctrl,
                                kl_penalty=self.config.algorithm.kl_penalty
                            )

                            # メトリクスにKL関連のメトリクスを追加
                            metrics.update(kl_metrics)
                        else:
                            # KLペナルティを適用しない場合、KL係数を0に設定
                            batch.batch['token_level_rewards'] = batch.batch['token_level_scores']

                        # アドバンテージを計算
                        batch = compute_advantage(
                            batch,
                            adv_estimator=self.config.algorithm.adv_estimator,
                            gamma=self.config.algorithm.gamma,
                            lam=self.config.algorithm.lam,
                            num_repeat=self.config.actor_rollout_ref.rollout.n
                        )

                    # クリティックを使用する場合
                    if self.use_critic:

                        with _timer('update_critic', timing_raw):
                            # クリティックを更新
                            critic_output = self.critic_wg.update_critic(batch)

                        # クリティックの出力メトリクスを集約
                        critic_output_metrics = reduce_metrics(critic_output.meta_info['metrics'])

                        # メトリクスに追加
                        metrics.update(critic_output_metrics)

                    # アクターのウォームアップ期間が終了した場合
                    if self.config.trainer.critic_warmup <= self.global_steps:

                        with _timer('update_actor', timing_raw):
                            # アクターを更新
                            actor_output = self.actor_rollout_wg.update_actor(batch)

                        # アクターの出力メトリクスを集約
                        actor_output_metrics = reduce_metrics(actor_output.meta_info['metrics'])

                        # メトリクスに追加
                        metrics.update(actor_output_metrics)

                    # 検証
                    if self.val_reward_fn is not None and self.config.trainer.test_freq > 0 and \
                        self.global_steps % self.config.trainer.test_freq == 0:

                        with _timer('testing', timing_raw):
                            # 検証データセットで評価
                            val_metrics: dict = self._validate()

                        # 検証メトリクスを追加
                        metrics.update(val_metrics)

                    # チェックポイントの保存
                    if self.config.trainer.save_freq > 0 and \
                            self.global_steps % self.config.trainer.save_freq == 0:

                        with _timer('save_checkpoint', timing_raw):
                            # チェックポイントを保存
                            self._save_checkpoint()

                # データに関するメトリクスを計算
                metrics.update(compute_data_metrics(batch=batch, use_critic=self.use_critic))

                # 処理時間に関するメトリクスを計算
                metrics.update(compute_timing_metrics(batch=batch, timing_raw=timing_raw))

                # ログにメトリクスを記録
                logger.log(data=metrics, step=self.global_steps)

                # グローバルステップをインクリメント
                self.global_steps += 1

                # 総訓練ステップ数に達した場合
                if self.global_steps >= self.total_training_steps:

                    # 検証用の報酬関数が存在する場合
                    if self.val_reward_fn is not None:

                        # 検証データセットで最終評価
                        val_metrics = self._validate()
                        
                        pprint(f'最終的な検証メトリクス: {val_metrics}')

                        logger.log(data=val_metrics, step=self.global_steps)

                    # 学習ループを終了
                    return

### 報酬関数

In [ ]:
def extract_solution(solution_str):
    """Extract the equation from the solution string."""
    # Remove everything before the first "Assistant:"
    if "Assistant:" in solution_str:
        solution_str = solution_str.split("Assistant:", 1)[1]
    elif "<|im_start|>assistant" in solution_str:
        solution_str = solution_str.split("<|im_start|>assistant", 1)[1]
    else:
        return None
    solution_str = solution_str.split('\n')[-1]

    answer_pattern = r'<answer>(.*?)</answer>'
    match = re.finditer(answer_pattern, solution_str)
    matches = list(match)
    if matches:
        final_answer = matches[-1].group(1).strip()
    else:
        final_answer = None
    return final_answer

In [ ]:
def validate_equation(equation_str, available_numbers):
    """Validate that equation only uses available numbers and each number once."""
    try:
        # Extract all numbers from the equation
        numbers_in_eq = [int(n) for n in re.findall(r'\d+', equation_str)]
        
        # Check if all numbers in equation are available
        available_numbers = sorted(available_numbers)
        numbers_in_eq = sorted(numbers_in_eq)
        
        # Each number should be used exactly once
        return numbers_in_eq == available_numbers
    except:
        return False

In [ ]:
def evaluate_equation(equation_str):
    """Safely evaluate the arithmetic equation using eval() with precautions."""
    try:
        # Define a regex pattern that only allows numbers, operators, parentheses, and whitespace
        allowed_pattern = r'^[\d+\-*/().\s]+$'
        if not re.match(allowed_pattern, equation_str):
            raise ValueError("Invalid characters in equation.")

        # Evaluate the equation with restricted globals and locals
        result = eval(equation_str, {"__builtins__": None}, {})
        return result
    except Exception as e:
        return None

In [ ]:
def compute_score(solution_str, ground_truth, method='strict', format_score=0.1, score=1.):
    """The scoring function for countdown task.
    
    Args:
        solution_str: the solution text
        ground_truth: dictionary containing target number and available numbers
        method: the method to extract the solution
        format_score: the score for correct format but wrong answer
        score: the score for the correct answer
    """
    target = ground_truth['target']
    numbers = ground_truth['numbers']
    
    equation = extract_solution(solution_str=solution_str)
    do_print = random.randint(1, 64) == 1
    
    if do_print:
        print(f"--------------------------------")
        print(f"Target: {target} | Numbers: {numbers}")
        print(f"Extracted equation: {equation}")
        print(f"Solution string: {solution_str}")

    if equation is None:
        if do_print:
            print(f"No equation found")
        return 0
    
    # Validate equation uses correct numbers
    if not validate_equation(equation, numbers):
        if do_print:
            print(f"Invalid equation")
        return format_score
        
    # Evaluate equation
    try:
        result = evaluate_equation(equation)
        if result is None:
            if do_print:
                print(f"Could not evaluate equation")
            return format_score
            
        if abs(result - target) < 1e-5:  # Account for floating point precision
            if do_print:
                print(f"Correct equation: {equation} = {result}")
            return score
        else:
            if do_print:
                print(f"Wrong result: equation = {result}, target = {target}")
            return format_score
    except:
        if do_print:
            print(f"Error evaluating equation")
        return format_score 

## 訓練

```sh
N_GPUS=1 BASE_MODEL="qwen/qwen2.5-0.5b" DATA_DIR="countdown" ROLLOUT_TP_SIZE=1 VLLM_ATTENTION_BACKEND="XFORMERS" bash ./scripts/train_tiny_zero.sh
```

In [ ]:
# コマンドラインの環境変数を変数化

BASE_MODEL="qwen/qwen2.5-0.5b"
DATA_DIR = "countdown"
EXPERIMENT_NAME = "countdown_ppo_demo"
N_GPUS = 1
ROLLOUT_TP_SIZE = 1
VLLM_ATTENTION_BACKEND = "XFORMERS"

In [ ]:
# PPOトレーナーのデフォルト設定を読み込み

# TinyZero/verl/trainer/config/ppo_trainer.yaml
ppo_trainer_yaml = """
data:
  tokenizer: null
  train_files: ~/data/rlhf/gsm8k/train.parquet
  val_files: ~/data/rlhf/gsm8k/test.parquet
  prompt_key: prompt
  max_prompt_length: 512
  max_response_length: 512
  train_batch_size: 1024
  val_batch_size: 1312
  return_raw_input_ids: False  # This should be set to true when the tokenizer between policy and rm differs
  return_raw_chat: False

actor_rollout_ref:
  hybrid_engine: True
  model:
    path: ~/models/deepseek-llm-7b-chat
    external_lib: null
    override_config: { }
    enable_gradient_checkpointing: False
    use_remove_padding: False
  actor:
    strategy: fsdp  # This is for backward-compatibility
    ppo_mini_batch_size: 256
    ppo_micro_batch_size: 64
    use_dynamic_bsz: False
    ppo_max_token_len_per_gpu: 16384 # n * ${data.max_prompt_length} + ${data.max_response_length}
    grad_clip: 1.0
    clip_ratio: 0.2
    entropy_coeff: 0.001
    use_kl_loss: False # True for GRPO
    kl_loss_coef: 0.001 # for grpo
    kl_loss_type: low_var_kl # for grpo
    ppo_epochs: 1
    shuffle: False
    ulysses_sequence_parallel_size: 1 # sp size
    optim:
      lr: 1e-6
      lr_warmup_steps_ratio: 0.  # the total steps will be injected during runtime
      min_lr_ratio: null   # only useful for warmup with cosine
      warmup_style: constant  # select from constant/cosine
      total_training_steps: -1  # must be override by program
    fsdp_config:
      wrap_policy:
        # transformer_layer_cls_to_wrap: None
        min_num_params: 0
      param_offload: False
      grad_offload: False
      optimizer_offload: False
      fsdp_size: -1
  ref:
    fsdp_config:
      param_offload: False
      wrap_policy:
        # transformer_layer_cls_to_wrap: None
        min_num_params: 0
      fsdp_size: -1
    log_prob_micro_batch_size: 128
    log_prob_use_dynamic_bsz: ${actor_rollout_ref.actor.use_dynamic_bsz}
    log_prob_max_token_len_per_gpu: ${actor_rollout_ref.actor.ppo_max_token_len_per_gpu}
    ulysses_sequence_parallel_size: ${actor_rollout_ref.actor.ulysses_sequence_parallel_size} # sp size
  rollout:
    name: vllm
    temperature: 1.0
    top_k: -1 # 0 for hf rollout, -1 for vllm rollout
    top_p: 1
    prompt_length: ${data.max_prompt_length}  # not use for opensource
    response_length: ${data.max_response_length}
    # for vllm rollout
    dtype: bfloat16 # should align with FSDP
    gpu_memory_utilization: 0.5
    ignore_eos: False
    enforce_eager: True
    free_cache_engine: True
    load_format: dummy_dtensor
    tensor_model_parallel_size: 2
    max_num_batched_tokens: 8192
    max_num_seqs: 1024
    log_prob_micro_batch_size: 128
    log_prob_use_dynamic_bsz: ${actor_rollout_ref.actor.use_dynamic_bsz}
    log_prob_max_token_len_per_gpu: ${actor_rollout_ref.actor.ppo_max_token_len_per_gpu}
    # for hf rollout
    do_sample: True
    # number of responses (i.e. num sample times)
    n: 1 # > 1 for grpo

critic:
  strategy: fsdp
  optim:
    lr: 1e-5
    lr_warmup_steps_ratio: 0.  # the total steps will be injected during runtime
    min_lr_ratio: null   # only useful for warmup with cosine
    warmup_style: constant  # select from constant/cosine
    total_training_steps: -1  # must be override by program
  model:
    path: ~/models/deepseek-llm-7b-chat
    tokenizer_path: ${actor_rollout_ref.model.path}
    override_config: { }
    external_lib: ${actor_rollout_ref.model.external_lib}
    enable_gradient_checkpointing: False
    use_remove_padding: False
    fsdp_config:
      param_offload: False
      grad_offload: False
      optimizer_offload: False
      wrap_policy:
        # transformer_layer_cls_to_wrap: None
        min_num_params: 0
      fsdp_size: -1
  ppo_mini_batch_size: ${actor_rollout_ref.actor.ppo_mini_batch_size}
  ppo_micro_batch_size: 64
  forward_micro_batch_size: ${critic.ppo_micro_batch_size}
  use_dynamic_bsz: ${actor_rollout_ref.actor.use_dynamic_bsz}
  ppo_max_token_len_per_gpu: 32768 # (${actor_rollout_ref.actor.ppo_max_token_len_per_gpu}) * 2
  forward_max_token_len_per_gpu: ${critic.ppo_max_token_len_per_gpu}
  ulysses_sequence_parallel_size: 1 # sp size
  ppo_epochs: ${actor_rollout_ref.actor.ppo_epochs}
  shuffle: ${actor_rollout_ref.actor.shuffle}
  grad_clip: 1.0
  cliprange_value: 0.5

reward_model:
  enable: False
  strategy: fsdp
  model:
    input_tokenizer: ${actor_rollout_ref.model.path}  # set this to null if the chat template is identical
    path: ~/models/FsfairX-LLaMA3-RM-v0.1
    external_lib: ${actor_rollout_ref.model.external_lib}
    use_remove_padding: False
    fsdp_config:
      min_num_params: 0
      param_offload: False
  micro_batch_size: 64
  max_length: null
  ulysses_sequence_parallel_size: 1 # sp size
  use_dynamic_bsz: ${critic.use_dynamic_bsz}
  forward_max_token_len_per_gpu: ${critic.forward_max_token_len_per_gpu}

algorithm:
  gamma: 1.0
  lam: 1.0
  adv_estimator: gae
  kl_penalty: kl  # how to estimate kl divergence
  kl_ctrl:
    type: fixed
    kl_coef: 0.001

trainer:
  total_epochs: 30
  total_training_steps: null
  project_name: verl_examples
  experiment_name: gsm8k
  logger: [ 'console', 'wandb' ]
  nnodes: 1
  n_gpus_per_node: 8
  save_freq: -1
  test_freq: -1
  critic_warmup: 0
  default_hdfs_dir: ~/experiments/gsm8k/ppo/${trainer.experiment_name}
  default_local_dir: checkpoints/${trainer.project_name}/${trainer.experiment_name}
"""

ppo_trainer_config = OmegaConf.create(ppo_trainer_yaml)

In [ ]:
# PPOトレーニングのデフォルト設定を上書き

# TinyZero/scripts/train_tiny_zero.shより抜粋
# python3 -m verl.trainer.main_ppo \
# data.train_files=$DATA_DIR/train.parquet \
# data.val_files=$DATA_DIR/test.parquet \
# data.train_batch_size=32 \
# data.val_batch_size=32 \
# data.max_prompt_length=256 \
# data.max_response_length=1024 \
# actor_rollout_ref.model.path=$BASE_MODEL \
# actor_rollout_ref.model.use_remove_padding=True \
# actor_rollout_ref.model.enable_gradient_checkpointing=True \
# actor_rollout_ref.actor.use_dynamic_bsz=True \
# actor_rollout_ref.actor.optim.lr=1e-6 \
# actor_rollout_ref.actor.ppo_mini_batch_size=4 \
# actor_rollout_ref.actor.ppo_micro_batch_size=4 \
# actor_rollout_ref.rollout.log_prob_micro_batch_size=1 \
# actor_rollout_ref.rollout.tensor_model_parallel_size=$ROLLOUT_TP_SIZE \
# actor_rollout_ref.rollout.gpu_memory_utilization=0.1 \
# actor_rollout_ref.ref.log_prob_micro_batch_size=1 \
# critic.optim.lr=1e-5 \
# critic.model.path=$BASE_MODEL \
# critic.ppo_micro_batch_size=1 \
# algorithm.kl_ctrl.kl_coef=0.001 \
# trainer.logger=[] \
# +trainer.val_before_train=False \
# trainer.default_hdfs_dir=null \
# trainer.n_gpus_per_node=$N_GPUS \
# trainer.nnodes=1 \
# trainer.save_freq=100 \
# trainer.test_freq=100 \
# trainer.project_name=TinyZero \
# trainer.experiment_name=$EXPERIMENT_NAME \
# trainer.total_epochs=15 2>&1 | tee verl_demo.log

ppo_trainer_config.data.train_files = os.path.join(DATA_DIR, "train.parquet")
ppo_trainer_config.data.val_files = os.path.join(DATA_DIR, "test.parquet")
ppo_trainer_config.data.train_batch_size = 32
ppo_trainer_config.data.val_batch_size = 32
ppo_trainer_config.data.max_prompt_length = 256
ppo_trainer_config.data.max_response_length = 1024
ppo_trainer_config.actor_rollout_ref.model.path = BASE_MODEL
ppo_trainer_config.actor_rollout_ref.model.use_remove_padding = True
ppo_trainer_config.actor_rollout_ref.model.enable_gradient_checkpointing = True
ppo_trainer_config.actor_rollout_ref.actor.use_dynamic_bsz = True
ppo_trainer_config.actor_rollout_ref.actor.optim.lr = 1e-6
ppo_trainer_config.actor_rollout_ref.actor.ppo_mini_batch_size = 4
ppo_trainer_config.actor_rollout_ref.actor.ppo_micro_batch_size = 4
ppo_trainer_config.actor_rollout_ref.rollout.log_prob_micro_batch_size = 1
ppo_trainer_config.actor_rollout_ref.rollout.tensor_model_parallel_size = ROLLOUT_TP_SIZE
ppo_trainer_config.actor_rollout_ref.rollout.gpu_memory_utilization = 0.1
ppo_trainer_config.actor_rollout_ref.ref.log_prob_micro_batch_size = 1
ppo_trainer_config.critic.optim.lr = 1e-5
ppo_trainer_config.critic.model.path = BASE_MODEL
ppo_trainer_config.critic.ppo_micro_batch_size = 1
ppo_trainer_config.algorithm.kl_ctrl.kl_coef = 0.001
ppo_trainer_config.trainer.logger = []
ppo_trainer_config.trainer.val_before_train = False
ppo_trainer_config.trainer.default_hdfs_dir = None
ppo_trainer_config.trainer.n_gpus_per_node = N_GPUS
ppo_trainer_config.trainer.nnodes = 1
ppo_trainer_config.trainer.save_freq = 100
ppo_trainer_config.trainer.test_freq = 100
ppo_trainer_config.trainer.project_name = "TinyZero"
ppo_trainer_config.trainer.experiment_name = EXPERIMENT_NAME

# デバッグ目的でエポック数を1に設定
ppo_trainer_config.trainer.total_epochs = 1

In [ ]:
def _select_rm_score_fn(data_source):
    # if data_source == 'openai/gsm8k':
    #     return gsm8k.compute_score
    # elif data_source == 'lighteval/MATH':
    #     return math.compute_score
    # elif "multiply" in data_source or "arithmetic" in data_source:
    #     return multiply.compute_score
    # elif "countdown" in data_source:
    #     return countdown.compute_score
    # else:
    #     raise NotImplementedError
    return compute_score

In [ ]:
class RewardManager():
    """The reward manager.
    """

    def __init__(self, tokenizer, num_examine) -> None:
        self.tokenizer = tokenizer
        self.num_examine = num_examine  # the number of batches of decoded responses to print to the console

    def __call__(self, data: DataProto):
        """We will expand this function gradually based on the available datasets"""

        # If there is rm score, we directly return rm score. Otherwise, we compute via rm_score_fn
        if 'rm_scores' in data.batch.keys():
            return data.batch['rm_scores']

        reward_tensor = torch.zeros_like(data.batch['responses'], dtype=torch.float32)

        already_print_data_sources = {}

        for i in range(len(data)):
            data_item = data[i]  # DataProtoItem

            prompt_ids = data_item.batch['prompts']

            prompt_length = prompt_ids.shape[-1]

            valid_prompt_length = data_item.batch['attention_mask'][:prompt_length].sum()
            valid_prompt_ids = prompt_ids[-valid_prompt_length:]

            response_ids = data_item.batch['responses']
            valid_response_length = data_item.batch['attention_mask'][prompt_length:].sum()
            valid_response_ids = response_ids[:valid_response_length]

            # decode
            sequences = torch.cat((valid_prompt_ids, valid_response_ids))
            sequences_str = self.tokenizer.decode(sequences)

            ground_truth = data_item.non_tensor_batch['reward_model']['ground_truth']

            # select rm_score
            data_source = data_item.non_tensor_batch['data_source']
            compute_score_fn = _select_rm_score_fn(data_source)

            score = compute_score_fn(solution_str=sequences_str, ground_truth=ground_truth)
            reward_tensor[i, valid_response_length - 1] = score

            if data_source not in already_print_data_sources:
                already_print_data_sources[data_source] = 0

            if already_print_data_sources[data_source] < self.num_examine:
                already_print_data_sources[data_source] += 1
                print(sequences_str)

        return reward_tensor

In [ ]:
# @ray.remote
def main_task(config):
    logger.info(f"メインタスクを開始")

    # from verl.utils.fs import copy_local_path_from_hdfs
    from transformers import AutoTokenizer

    # 1) 初期化

    # 1-1) 設定を表示
    # pprint(OmegaConf.to_container(config, resolve=True))
    OmegaConf.resolve(config)
    logger.debug(f"設定: {config}")

    # 1-2) Hadoop File System (HDFS) からチェックポイントをダウンロード
    local_path = copy_local_path_from_hdfs(config.actor_rollout_ref.model.path)
    logger.debug(f"モデルのローカルパス: {local_path}")

    # 1-3) トークナイザーの初期化
    tokenizer = hf_tokenizer(local_path)
    logger.debug(f"トークナイザー: {tokenizer}")

    # 1-4) Rayのワーカーグループの初期化
    if config.actor_rollout_ref.actor.strategy == 'fsdp':
        logger.debug("FSDPワーカーグループを使用")
        assert config.actor_rollout_ref.actor.strategy == config.critic.strategy
        # from verl.workers.fsdp_workers import ActorRolloutRefWorker, CriticWorker
        # from verl.single_controller.ray import RayWorkerGroup
        ray_worker_group_cls = RayWorkerGroup

    # elif config.actor_rollout_ref.actor.strategy == 'megatron':
    #     logger.debug("Megatronワーカーグループを使用")
    #     assert config.actor_rollout_ref.actor.strategy == config.critic.strategy
    #     from verl.workers.megatron_workers import ActorRolloutRefWorker, CriticWorker
    #     from verl.single_controller.ray.megatron import NVMegatronRayWorkerGroup
    #     ray_worker_group_cls = NVMegatronRayWorkerGroup

    else:
        raise NotImplementedError

    # from verl.trainer.ppo.ray_trainer import ResourcePoolManager, Role

    role_worker_mapping = {
        Role.ActorRollout: ray.remote(ActorRolloutRefWorker),
        Role.Critic: ray.remote(CriticWorker),
        Role.RefPolicy: ray.remote(ActorRolloutRefWorker)
    }

    global_pool_id = 'global_pool'

    resource_pool_spec = {
        global_pool_id: [config.trainer.n_gpus_per_node] * config.trainer.nnodes,
    }

    mapping = {
        Role.ActorRollout: global_pool_id,
        Role.Critic: global_pool_id,
        Role.RefPolicy: global_pool_id,
    }

    # we should adopt a multi-source reward function here
    # - for rule-based rm, we directly call a reward score
    # - for model-based rm, we call a model
    # - for code related prompt, we send to a sandbox if there are test cases
    # - finally, we combine all the rewards together
    # - The reward type depends on the tag of the data
    if config.reward_model.enable:
        if config.reward_model.strategy == 'fsdp':
            logger.debug("FSDP報酬モデルワーカーを使用")
            # from verl.workers.fsdp_workers import RewardModelWorker
        # elif config.reward_model.strategy == 'megatron':
        #     logger.debug("Megatron報酬モデルワーカーを使用")
        #     from verl.workers.megatron_workers import RewardModelWorker
        else:
            raise NotImplementedError

        role_worker_mapping[Role.RewardModel] = ray.remote(RewardModelWorker)

        mapping[Role.RewardModel] = global_pool_id
    else:
        logger.debug("関数ベースの報酬マネージャーを使用")

    reward_fn = RewardManager(tokenizer=tokenizer, num_examine=0)

    # Note that we always use function-based RM for validation
    val_reward_fn = RewardManager(tokenizer=tokenizer, num_examine=1)

    resource_pool_manager = ResourcePoolManager(
        resource_pool_spec=resource_pool_spec,
        mapping=mapping
    )

    trainer = RayPPOTrainer(
        config=config,
        tokenizer=tokenizer,
        role_worker_mapping=role_worker_mapping,
        resource_pool_manager=resource_pool_manager,
        ray_worker_group_cls=ray_worker_group_cls,
        reward_fn=reward_fn,
        val_reward_fn=val_reward_fn
    )

    trainer.init_workers()

    trainer.fit()

In [ ]:
# 訓練を開始

# @hydra.main(config_path='config', config_name='ppo_trainer', version_base=None)
def main(config):

    # Rayが初期化されていない場合
    if not ray.is_initialized():
        # ローカルのRayクラスターを初期化
        logger.debug(f"Rayを初期化")
        ray.init(runtime_env={
            'env_vars': {'TOKENIZERS_PARALLELISM': 'true', 'NCCL_DEBUG': 'WARN'}
        })

    # タスクを実行
    # ray.get(main_task.remote(config))
    main_task(config)

main(ppo_trainer_config)